# 6th_2014

## Raw Data

### 1

In [1]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_6th_governor_election1(
    file_path_or_url,
    header_rows=(2, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
    filter_column=None,
    filter_value=None
):
    """
    선거 데이터를 유연하게 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (2, 4) = 2행부터 4행까지
    filter_column : str, optional
        필터링할 컬럼명 (예: '읍면동명')
    filter_value : str, optional
        필터링할 값 (예: '합계')

    Returns:
    --------
    pandas.DataFrame
        처리된 데이터프레임

    Example:
    --------
    # 2~4행을 헤더로, '읍면동명'이 '합계'인 행만 추출
    df = process_6th_governor_election1(
        'https://github.com/.../강원도지사선거.xlsx',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value='합계'
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 파일을 header 없이 읽기
    df_raw = pd.read_excel(file_path_or_url, header=None)

    # 엑셀 행 번호를 파이썬 인덱스로 변환 (엑셀은 1부터, 파이썬은 0부터)
    start_idx = header_rows[0] - 1
    end_idx = header_rows[1] - 1

    # 지정된 행들을 가져와서 컬럼명 생성
    header_rows_data = []
    for i in range(start_idx, end_idx + 1):
        header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

    # 컬럼명 생성
    new_columns = []
    for col_idx in range(len(header_rows_data[0])):
        parts = []

        # 각 행의 값이 '_'가 아닌 경우만 추가
        for row_data in header_rows_data:
            if row_data[col_idx] != '_':
                parts.append(row_data[col_idx])

        # parts가 비어있으면 '_', 아니면 '_'로 연결
        if parts:
            new_columns.append('_'.join(parts))
        else:
            new_columns.append('_')

    # 데이터프레임 재구성 (헤더 다음 행부터 데이터로 사용)
    data_start_idx = end_idx + 1
    df = df_raw.iloc[data_start_idx:].copy()
    df.columns = new_columns
    df = df.reset_index(drop=True)

    print(f"헤더 행: {header_rows[0]}행 ~ {header_rows[1]}행")
    print(f"생성된 컬럼 수: {len(new_columns)}")
    print(f"데이터 행 수: {len(df)}")

    # 컬럼명 샘플 출력
    print("\n생성된 컬럼명 (처음 10개):")
    for i, col in enumerate(new_columns[:10]):
        print(f"{i}: {col}")

    # 필터링 적용
    if filter_column and filter_value:
        # 필터링할 컬럼 찾기
        matching_col = None
        for col in df.columns:
            if filter_column in col:
                matching_col = col
                break

        if matching_col:
            print(f"\n'{filter_column}' 컬럼 발견: {matching_col}")

            # 필터링 적용
            filtered_df = df[df[matching_col] == filter_value].copy()

            print(f"필터링 전: {len(df)}행 → 필터링 후: {len(filtered_df)}행")
            print(f"'{matching_col}' == '{filter_value}'인 행만 추출")

            return filtered_df
        else:
            print(f"\n경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")
            print("필터링 없이 전체 데이터를 반환합니다.")

    return df


# 사용 예시

# 1. 강원도지사선거 - 2~4행을 헤더로, '읍면동명'이 '합 계'인 행만
# url = "https://github.com/.../강원도지사선거.xlsx"
# summary_df = process_6th_governor_election1(
#     url,
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value='합 계'
# )

# 2. 다른 파일 - 1~3행을 헤더로, 필터링 없이
# df_all = process_6th_governor_election1(
#     'other_file.xlsx',
#     header_rows=(1, 3)
# )

# 3. 로컬 파일 - 기본값 사용
# df = process_6th_governor_election1('강원도지사선거.xlsx')

### 2

In [2]:
import pandas as pd
import numpy as np

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_6th_governor_election2(
    file_path_or_url,
    header_rows=(2, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
    filter_column=None,
    filter_value=None
):
    """
    선거 데이터를 유연하게 처리하는 함수 (복수 필터값 지원)

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (2, 4) = 2행부터 4행까지
    filter_column : str, optional
        필터링할 컬럼명 (예: '읍면동명')
    filter_value : str, list, or None, optional
        필터링할 값 또는 값들의 리스트
        - str: 단일 값 (예: '합계')
        - list: 복수 값 (예: [np.nan, '합계'])
        - None: NaN 값만 필터링

    Returns:
    --------
    pandas.DataFrame
        처리된 데이터프레임

    Example:
    --------
    # NaN과 '합계' 모두 포함
    df = process_6th_governor_election2(
        '경상북도.xlsx',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value=[np.nan, '합계']
    )

    # NaN 값만 포함
    df = process_6th_governor_election2(
        '경상북도.xlsx',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value=None
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 파일을 header 없이 읽기
    df_raw = pd.read_excel(file_path_or_url, header=None)

    # 엑셀 행 번호를 파이썬 인덱스로 변환 (엑셀은 1부터, 파이썬은 0부터)
    start_idx = header_rows[0] - 1
    end_idx = header_rows[1] - 1

    # 지정된 행들을 가져와서 컬럼명 생성
    header_rows_data = []
    for i in range(start_idx, end_idx + 1):
        header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

    # 컬럼명 생성
    new_columns = []
    for col_idx in range(len(header_rows_data[0])):
        parts = []

        # 각 행의 값이 '_'가 아닌 경우만 추가
        for row_data in header_rows_data:
            if row_data[col_idx] != '_':
                parts.append(row_data[col_idx])

        # parts가 비어있으면 '_', 아니면 '_'로 연결
        if parts:
            new_columns.append('_'.join(parts))
        else:
            new_columns.append('_')

    # 데이터프레임 재구성 (헤더 다음 행부터 데이터로 사용)
    data_start_idx = end_idx + 1
    df = df_raw.iloc[data_start_idx:].copy()
    df.columns = new_columns
    df = df.reset_index(drop=True)

    print(f"헤더 행: {header_rows[0]}행 ~ {header_rows[1]}행")
    print(f"생성된 컬럼 수: {len(new_columns)}")
    print(f"데이터 행 수: {len(df)}")

    # 컬럼명 샘플 출력
    print("\n생성된 컬럼명 (처음 10개):")
    for i, col in enumerate(new_columns[:10]):
        print(f"{i}: {col}")

    # 필터링 적용
    if filter_column is not None:
        # 필터링할 컬럼 찾기
        matching_col = None
        for col in df.columns:
            if filter_column in col:
                matching_col = col
                break

        if matching_col:
            print(f"\n'{filter_column}' 컬럼 발견: {matching_col}")

            # filter_value 처리
            if filter_value is None:
                # None인 경우 NaN만 필터링
                filtered_df = df[df[matching_col].isna()].copy()
                print(f"필터링: NaN 값만 추출")
            elif isinstance(filter_value, list):
                # 리스트인 경우 복수 값 필터링
                # NaN 값과 일반 값을 분리해서 처리
                conditions = []
                non_nan_values = []

                for val in filter_value:
                    if pd.isna(val):
                        conditions.append(df[matching_col].isna())
                    else:
                        non_nan_values.append(val)

                # 일반 값들에 대한 조건 추가
                if non_nan_values:
                    conditions.append(df[matching_col].isin(non_nan_values))

                # 모든 조건을 OR로 결합
                if conditions:
                    combined_condition = conditions[0]
                    for condition in conditions[1:]:
                        combined_condition = combined_condition | condition
                    filtered_df = df[combined_condition].copy()
                else:
                    filtered_df = df.copy()

                print(f"필터링: {filter_value} 값들 중 하나인 행 추출")
            else:
                # 단일 값인 경우
                if pd.isna(filter_value):
                    filtered_df = df[df[matching_col].isna()].copy()
                else:
                    filtered_df = df[df[matching_col] == filter_value].copy()
                print(f"필터링: '{matching_col}' == '{filter_value}'인 행 추출")

            print(f"필터링 전: {len(df)}행 → 필터링 후: {len(filtered_df)}행")
            return filtered_df
        else:
            print(f"\n경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")
            print("필터링 없이 전체 데이터를 반환합니다.")

    return df


# 사용 예시

# 1. 경상북도.xlsx - NaN과 '합계' 모두 포함
# df_multi = process_6th_governor_election2(
#     '경상북도.xlsx',
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value=[np.nan, '합계']
# )

# 2. NaN 값만 필터링
# df_nan_only = process_6th_governor_election2(
#     '경상북도.xlsx',
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value=None  # None은 NaN만 필터링
# )

# 3. 여러 값 필터링 (NaN, '합계', '소계')
# df_multiple = process_6th_governor_election2(
#     '경상북도.xlsx',
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value=[np.nan, '합계', '소계']
# )

# 4. 단일 값 필터링 (기존 방식과 동일)
# df_single = process_6th_governor_election2(
#     '경상북도.xlsx',
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value='합계'
# )

### 3

In [3]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_6th_governor_election3(
    file_path_or_url,
    header_rows=(2, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
    filter_column=None,
    filter_value=None,
    add_sheet_name_column=True
):
    """
    여러 시트가 있는 선거 데이터를 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (2, 4) = 2행부터 4행까지
    filter_column : str, optional
        필터링할 컬럼명 (예: '읍면동명')
    filter_value : str, optional
        필터링할 값 (예: '합계')
    add_sheet_name_column : bool
        시트명을 첫 번째 열로 추가할지 여부 (기본값: True)

    Returns:
    --------
    pandas.DataFrame
        모든 시트를 합친 데이터프레임

    Example:
    --------
    # 모든 시트에서 2~4행을 헤더로, '읍면동명'이 '합계'인 행만 추출
    df = process_6th_governor_election3(
        'https://github.com/.../서울특별시.xls',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value='합계'
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 모든 시트 이름 가져오기
    excel_file = pd.ExcelFile(file_path_or_url)
    sheet_names = excel_file.sheet_names
    print(f"발견된 시트: {len(sheet_names)}개")
    print(f"시트 이름: {sheet_names}\n")

    # 각 시트의 결과를 저장할 리스트
    all_dfs = []

    # 각 시트 처리
    for sheet_idx, sheet_name in enumerate(sheet_names):
        print(f"\n{'='*50}")
        print(f"시트 {sheet_idx+1}/{len(sheet_names)}: '{sheet_name}' 처리 중...")
        print(f"{'='*50}")

        # 파일을 header 없이 읽기
        df_raw = pd.read_excel(file_path_or_url, sheet_name=sheet_name, header=None)

        # 빈 시트 체크
        if df_raw.empty or len(df_raw) == 0:
            print(f"'{sheet_name}' 시트가 비어있습니다. 건너뜁니다.")
            continue

        # 엑셀 행 번호를 파이썬 인덱스로 변환
        start_idx = header_rows[0] - 1
        end_idx = header_rows[1] - 1

        # 헤더 행이 충분한지 확인
        if len(df_raw) <= end_idx:
            print(f"'{sheet_name}' 시트의 행이 부족합니다. 건너뜁니다.")
            continue

        # 지정된 행들을 가져와서 컬럼명 생성
        header_rows_data = []
        for i in range(start_idx, end_idx + 1):
            header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

        # 컬럼명 생성
        new_columns = []
        for col_idx in range(len(header_rows_data[0])):
            parts = []

            # 각 행의 값이 '_'가 아닌 경우만 추가
            for row_data in header_rows_data:
                if row_data[col_idx] != '_':
                    parts.append(row_data[col_idx])

            # parts가 비어있으면 '_', 아니면 '_'로 연결
            if parts:
                new_columns.append('_'.join(parts))
            else:
                new_columns.append('_')

        # 데이터프레임 재구성
        data_start_idx = end_idx + 1
        df = df_raw.iloc[data_start_idx:].copy()
        df.columns = new_columns
        df = df.reset_index(drop=True)

        print(f"데이터 행 수: {len(df)}")

        # 필터링 적용
        if filter_column and filter_value:
            # 필터링할 컬럼 찾기
            matching_col = None
            for col in df.columns:
                if filter_column in col:
                    matching_col = col
                    break

            if matching_col:
                # 필터링 적용
                df = df[df[matching_col] == filter_value].copy()
                print(f"필터링 적용: '{matching_col}' == '{filter_value}' → {len(df)}행")
            else:
                print(f"경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")

        # 시트명 열 추가
        if add_sheet_name_column and len(df) > 0:
            df.insert(0, '시트명', sheet_name)

        # 결과 리스트에 추가
        if len(df) > 0:
            all_dfs.append(df)
            print(f"'{sheet_name}' 시트 처리 완료: {len(df)}행 추가됨")
        else:
            print(f"'{sheet_name}' 시트에 데이터가 없습니다.")

    # 모든 데이터프레임 합치기
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print(f"\n{'='*50}")
        print(f"최종 결과: 총 {len(combined_df)}행의 데이터")
        print(f"{'='*50}")

        return combined_df
    else:
        print("\n처리된 데이터가 없습니다.")
        return pd.DataFrame()


# 사용 예시

# 1. 서울특별시.xls - 모든 시트의 합계 데이터만 추출
# url = "https://github.com/.../서울특별시.xls"
# seoul_summary = process_6th_governor_election3(
#     url,
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value='합계'
# )

# 2. 시트명 열 없이 처리
# df_no_sheet = process_6th_governor_election3(
#     'multi_sheet.xlsx',
#     header_rows=(1, 3),
#     add_sheet_name_column=False
# )

# 3. 필터링 없이 모든 데이터 합치기
# df_all = process_6th_governor_election3(
#     'election_data.xls',
#     header_rows=(2, 2)  # 2행만 헤더로 사용
# )

### 4

In [4]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_6th_governor_election4(
    file_path_or_url,
    header_rows=(2, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
    filter_column=None,
    filter_value=None,
    row_range=None  # 읽을 행 범위 (엑셀 기준)
):
    """
    선거 데이터를 유연하게 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (2, 4) = 2행부터 4행까지
    filter_column : str, optional
        필터링할 컬럼명 (예: '읍면동명')
    filter_value : str, optional
        필터링할 값 (예: '합계')
    row_range : tuple, optional
        읽을 행 범위 (엑셀 기준 행 번호)
        예: (1, 13) = 1행부터 13행까지만 읽기
        None이면 전체 파일 읽기

    Returns:
    --------
    pandas.DataFrame
        처리된 데이터프레임

    Example:
    --------
    # 1~13행만 읽어서 2~4행을 헤더로, '읍면동명'이 '합계'인 행만 추출
    df = process_6th_governor_election4(
        '대구광역시.xls',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value='합계',
        row_range=(1, 13)
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 파일을 header 없이 읽기
    if row_range:
        # 행 범위가 지정된 경우
        start_row = row_range[0] - 1  # 엑셀 행 번호를 파이썬 인덱스로 변환
        end_row = row_range[1]  # 엑셀의 13행까지 = 파이썬의 인덱스 12까지
        nrows = end_row - start_row

        print(f"파일에서 {row_range[0]}행부터 {row_range[1]}행까지만 읽습니다.")
        df_raw = pd.read_excel(file_path_or_url, header=None, skiprows=start_row, nrows=nrows)

        # 헤더 인덱스 조정 (skiprows를 사용했으므로)
        adjusted_start_idx = header_rows[0] - row_range[0]
        adjusted_end_idx = header_rows[1] - row_range[0]
    else:
        # 전체 파일 읽기
        print("파일 전체를 읽습니다.")
        df_raw = pd.read_excel(file_path_or_url, header=None)
        adjusted_start_idx = header_rows[0] - 1
        adjusted_end_idx = header_rows[1] - 1

    # 지정된 행들을 가져와서 컬럼명 생성
    header_rows_data = []
    for i in range(adjusted_start_idx, adjusted_end_idx + 1):
        if i < 0 or i >= len(df_raw):
            print(f"경고: 헤더 행 인덱스 {i}가 범위를 벗어났습니다.")
            continue
        header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

    # 컬럼명 생성
    new_columns = []
    for col_idx in range(len(header_rows_data[0])):
        parts = []

        # 각 행의 값이 '_'가 아닌 경우만 추가
        for row_data in header_rows_data:
            if row_data[col_idx] != '_':
                parts.append(row_data[col_idx])

        # parts가 비어있으면 '_', 아니면 '_'로 연결
        if parts:
            new_columns.append('_'.join(parts))
        else:
            new_columns.append('_')

    # 데이터프레임 재구성 (헤더 다음 행부터 데이터로 사용)
    data_start_idx = adjusted_end_idx + 1
    if data_start_idx < len(df_raw):
        df = df_raw.iloc[data_start_idx:].copy()
        df.columns = new_columns
        df = df.reset_index(drop=True)
    else:
        print("경고: 데이터 행이 없습니다.")
        df = pd.DataFrame(columns=new_columns)

    print(f"헤더 행: {header_rows[0]}행 ~ {header_rows[1]}행")
    print(f"생성된 컬럼 수: {len(new_columns)}")
    print(f"데이터 행 수: {len(df)}")

    # 컬럼명 샘플 출력
    print("\n생성된 컬럼명 (처음 10개):")
    for i, col in enumerate(new_columns[:10]):
        print(f"{i}: {col}")

    # 필터링 적용
    if filter_column and filter_value:
        # 필터링할 컬럼 찾기
        matching_col = None
        for col in df.columns:
            if filter_column in col:
                matching_col = col
                break

        if matching_col:
            print(f"\n'{filter_column}' 컬럼 발견: {matching_col}")

            # 필터링 적용
            filtered_df = df[df[matching_col] == filter_value].copy()

            print(f"필터링 전: {len(df)}행 → 필터링 후: {len(filtered_df)}행")
            print(f"'{matching_col}' == '{filter_value}'인 행만 추출")

            return filtered_df
        else:
            print(f"\n경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")
            print("필터링 없이 전체 데이터를 반환합니다.")

    return df


# 사용 예시

# df = process_6th_governor_election4(
#     "https://github.com/.../대구광역시.xls",
#     header_rows=(3, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
#     filter_column=None,
#     filter_value=None,
#     row_range=(1,13)  # 읽을 행 범위 (엑셀 기준)
# )

### 5

In [5]:
import pandas as pd

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_6th_governor_election5(
    file_path_or_url,
    header_rows=(2, 4),  # 엑셀 기준 행 번호 (2행부터 4행까지)
    filter_column=None,
    filter_value=None,
    add_sheet_name_column=True,
    sheet_names=None  # 특정 시트만 처리하고 싶을 때 사용
):
    """
    여러 시트가 있는 선거 데이터를 처리하는 함수

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (2, 4) = 2행부터 4행까지
    filter_column : str, optional
        필터링할 컬럼명 (예: '읍면동명')
    filter_value : str, optional
        필터링할 값 (예: '합계')
    add_sheet_name_column : bool
        시트명을 첫 번째 열로 추가할지 여부 (기본값: True)
    sheet_names : str or list, optional
        처리할 시트명. None이면 모든 시트 처리
        - str: 단일 시트명 (예: '세종특별자치시장')
        - list: 여러 시트명 (예: ['세종특별자치시장', '세종시의원'])

    Returns:
    --------
    pandas.DataFrame
        지정된 시트들을 합친 데이터프레임

    Example:
    --------
    # 특정 시트만 처리
    df = process_6th_governor_election5(
        'election.xlsx',
        header_rows=(2, 4),
        filter_column='읍면동명',
        filter_value='합계',
        sheet_names='세종특별자치시장'  # 한 시트만
    )

    # 여러 시트 처리
    df = process_6th_governor_election5(
        'election.xlsx',
        sheet_names=['세종특별자치시장', '세종시의원']  # 두 시트만
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 모든 시트 이름 가져오기
    excel_file = pd.ExcelFile(file_path_or_url)
    all_sheet_names = excel_file.sheet_names
    print(f"파일의 전체 시트: {len(all_sheet_names)}개")
    print(f"시트 목록: {all_sheet_names}\n")

    # 처리할 시트 결정
    if sheet_names is None:
        # 모든 시트 처리
        sheets_to_process = all_sheet_names
        print("모든 시트를 처리합니다.")
    else:
        # 특정 시트만 처리
        if isinstance(sheet_names, str):
            sheets_to_process = [sheet_names]
        else:
            sheets_to_process = sheet_names

        # 존재하는 시트인지 확인
        invalid_sheets = [s for s in sheets_to_process if s not in all_sheet_names]
        if invalid_sheets:
            print(f"경고: 다음 시트가 존재하지 않습니다: {invalid_sheets}")
            sheets_to_process = [s for s in sheets_to_process if s in all_sheet_names]

        if not sheets_to_process:
            print("처리할 유효한 시트가 없습니다.")
            return pd.DataFrame()

        print(f"선택된 시트 {len(sheets_to_process)}개를 처리합니다: {sheets_to_process}")

    # 각 시트의 결과를 저장할 리스트
    all_dfs = []

    # 선택된 시트만 처리
    for sheet_idx, sheet_name in enumerate(sheets_to_process):
        print(f"\n{'='*50}")
        print(f"시트 {sheet_idx+1}/{len(sheets_to_process)}: '{sheet_name}' 처리 중...")
        print(f"{'='*50}")

        # 파일을 header 없이 읽기
        df_raw = pd.read_excel(file_path_or_url, sheet_name=sheet_name, header=None)

        # 빈 시트 체크
        if df_raw.empty or len(df_raw) == 0:
            print(f"'{sheet_name}' 시트가 비어있습니다. 건너뜁니다.")
            continue

        # 엑셀 행 번호를 파이썬 인덱스로 변환
        start_idx = header_rows[0] - 1
        end_idx = header_rows[1] - 1

        # 헤더 행이 충분한지 확인
        if len(df_raw) <= end_idx:
            print(f"'{sheet_name}' 시트의 행이 부족합니다. 건너뜁니다.")
            continue

        # 지정된 행들을 가져와서 컬럼명 생성
        header_rows_data = []
        for i in range(start_idx, end_idx + 1):
            header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

        # 컬럼명 생성
        new_columns = []
        for col_idx in range(len(header_rows_data[0])):
            parts = []

            # 각 행의 값이 '_'가 아닌 경우만 추가
            for row_data in header_rows_data:
                if row_data[col_idx] != '_':
                    parts.append(row_data[col_idx])

            # parts가 비어있으면 '_', 아니면 '_'로 연결
            if parts:
                new_columns.append('_'.join(parts))
            else:
                new_columns.append('_')

        # 데이터프레임 재구성
        data_start_idx = end_idx + 1
        df = df_raw.iloc[data_start_idx:].copy()
        df.columns = new_columns
        df = df.reset_index(drop=True)

        print(f"데이터 행 수: {len(df)}")

        # 필터링 적용
        if filter_column and filter_value:
            # 필터링할 컬럼 찾기
            matching_col = None
            for col in df.columns:
                if filter_column in col:
                    matching_col = col
                    break

            if matching_col:
                # 필터링 적용
                df = df[df[matching_col] == filter_value].copy()
                print(f"필터링 적용: '{matching_col}' == '{filter_value}' → {len(df)}행")
            else:
                print(f"경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")

        # 시트명 열 추가
        if add_sheet_name_column and len(df) > 0:
            df.insert(0, '시트명', sheet_name)

        # 결과 리스트에 추가
        if len(df) > 0:
            all_dfs.append(df)
            print(f"'{sheet_name}' 시트 처리 완료: {len(df)}행 추가됨")
        else:
            print(f"'{sheet_name}' 시트에 데이터가 없습니다.")

    # 모든 데이터프레임 합치기
    if all_dfs:
        combined_df = pd.concat(all_dfs, ignore_index=True)
        print(f"\n{'='*50}")
        print(f"최종 결과: 총 {len(combined_df)}행의 데이터")
        print(f"{'='*50}")

        return combined_df
    else:
        print("\n처리된 데이터가 없습니다.")
        return pd.DataFrame()


# 사용 예시

# 1. 특정 시트 하나만 처리
# sejong_mayor = process_6th_governor_election5(
#     '제6회 전국동시지방선거 읍면동별 개표자료세종.xlsx',
#     header_rows=(2, 4),
#     filter_column='읍면동명',
#     filter_value='합계',
#     sheet_names='세종특별자치시장'  # 시장 선거 데이터만
# )

# 2. 여러 특정 시트 처리
# sejong_elections = process_6th_governor_election5(
#     'sejong_election.xlsx',
#     header_rows=(2, 4),
#     sheet_names=['세종특별자치시장', '세종시의원', '세종교육감']
# )

# 3. 모든 시트 처리 (sheet_names=None 또는 생략)
# all_data = process_6th_governor_election5(
#     'election.xlsx',
#     header_rows=(2, 4)
# )

### 6

In [6]:
import pandas as pd
import numpy as np
import re

def convert_github_url_to_raw(github_url):
    """
    깃허브 blob URL을 raw URL로 변환

    Parameters:
    -----------
    github_url : str
        깃허브 파일 URL

    Returns:
    --------
    str
        raw URL (pandas가 직접 읽을 수 있는 형태)
    """
    if '/blob/' in github_url:
        return github_url.replace('/blob/', '/raw/')
    return github_url


def process_5th_governor_election6(
    file_path_or_url,
    header_rows=(3, 4),  # 엑셀 기준 행 번호 (3행부터 4행까지)
    filter_column='읍면동명',
    filter_value='합계',
    extract_district=True
):
    """
    제5회 선거 데이터를 처리하는 함수 (구시군명 자동 추출)

    Parameters:
    -----------
    file_path_or_url : str
        로컬 파일 경로 또는 GitHub URL
    header_rows : tuple
        헤더로 사용할 행 범위 (엑셀 기준 행 번호)
        예: (3, 4) = 3행부터 4행까지
    filter_column : str
        필터링할 컬럼명 (기본값: '읍면동명')
    filter_value : str
        필터링할 값 (기본값: '합계')
    extract_district : bool
        구시군명을 추출할지 여부 (기본값: True)

    Returns:
    --------
    pandas.DataFrame
        처리된 데이터프레임

    Example:
    --------
    # 부산시장개표진행상황읍면동별.xlsx 처리
    df = process_5th_governor_election6(
        '부산시장개표진행상황읍면동별.xlsx',
        header_rows=(3, 4),
        filter_column='읍면동명',
        filter_value='합계'
    )
    """

    # GitHub URL인 경우 raw URL로 변환
    if file_path_or_url.startswith('https://github.com'):
        file_path_or_url = convert_github_url_to_raw(file_path_or_url)

    # 파일을 header 없이 읽기
    df_raw = pd.read_excel(file_path_or_url, header=None)

    # 엑셀 행 번호를 파이썬 인덱스로 변환
    start_idx = header_rows[0] - 1
    end_idx = header_rows[1] - 1

    # 지정된 행들을 가져와서 컬럼명 생성
    header_rows_data = []
    for i in range(start_idx, end_idx + 1):
        header_rows_data.append(df_raw.iloc[i].fillna('_').astype(str))

    # 컬럼명 생성
    new_columns = []
    for col_idx in range(len(header_rows_data[0])):
        parts = []

        # 각 행의 값이 '_'가 아닌 경우만 추가
        for row_data in header_rows_data:
            if row_data[col_idx] != '_':
                parts.append(row_data[col_idx])

        # parts가 비어있으면 '_', 아니면 '_'로 연결
        if parts:
            new_columns.append('_'.join(parts))
        else:
            new_columns.append('_')

    # 데이터프레임 재구성 (헤더 다음 행부터 데이터로 사용)
    data_start_idx = end_idx + 1
    df = df_raw.iloc[data_start_idx:].copy()
    df.columns = new_columns
    df = df.reset_index(drop=True)

    print(f"헤더 행: {header_rows[0]}행 ~ {header_rows[1]}행")
    print(f"생성된 컬럼 수: {len(new_columns)}")
    print(f"데이터 행 수: {len(df)}")

    # 필터링할 컬럼 찾기
    matching_col = None
    for col in df.columns:
        if filter_column in col:
            matching_col = col
            break

    if not matching_col:
        print(f"\n경고: '{filter_column}'을 포함하는 컬럼을 찾을 수 없습니다.")
        return df

    print(f"\n'{filter_column}' 컬럼 발견: {matching_col}")

    # 구시군명 추출을 위한 준비
    if extract_district:
        # 원본 데이터에서 구시군명 추출
        district_info = {}

        # 전체 원본 데이터에서 패턴 찾기
        for idx in range(len(df_raw)):
            cell_value = str(df_raw.iloc[idx, 0])  # 첫 번째 열 확인

            # [지역][구분][구시군명] 패턴 찾기
            pattern = r'\[([^\]]+)\]\[([^\]]+)\]\[([^\]]+)\]'
            match = re.match(pattern, cell_value)

            if match:
                current_district = match.group(3)  # 세 번째 대괄호 내용
                # 해당 행 이후의 '합계' 행들에 적용
                for future_idx in range(idx + 1, min(idx + 50, len(df_raw))):
                    if future_idx - data_start_idx >= 0 and future_idx - data_start_idx < len(df):
                        district_info[future_idx - data_start_idx] = current_district

    # 필터링 적용
    filtered_indices = df[df[matching_col] == filter_value].index.tolist()
    filtered_df = df[df[matching_col] == filter_value].copy()

    # 구시군명 열 추가
    if extract_district and filtered_indices:
        district_names = []

        for idx in filtered_indices:
            if idx in district_info:
                district_names.append(district_info[idx])
            else:
                # district_info에 없는 경우 해당 행 위의 패턴 직접 찾기
                found = False
                for back_idx in range(idx + data_start_idx - 1, max(0, idx + data_start_idx - 10), -1):
                    cell_value = str(df_raw.iloc[back_idx, 0])
                    pattern = r'\[([^\]]+)\]\[([^\]]+)\]\[([^\]]+)\]'
                    match = re.match(pattern, cell_value)
                    if match:
                        district_names.append(match.group(3))
                        found = True
                        break

                if not found:
                    district_names.append('알수없음')

        # 구시군명 열을 맨 앞에 추가
        filtered_df.insert(0, '구시군명', district_names)
        print(f"\n구시군명 열 추가 완료")

    print(f"\n필터링 전: {len(df)}행 → 필터링 후: {len(filtered_df)}행")
    print(f"'{matching_col}' == '{filter_value}'인 행만 추출")

    # 첫 몇 행 출력하여 확인
    if len(filtered_df) > 0:
        print("\n결과 데이터 샘플:")
        print(filtered_df.head())

    return filtered_df


# 사용 예시

# 1. 부산시장개표진행상황읍면동별.xlsx 처리
# busan_df = process_5th_governor_election6(
#     '부산시장개표진행상황읍면동별.xlsx',
#     header_rows=(3, 4),
#     filter_column='읍면동명',
#     filter_value='합계'
# )

# 2. 구시군명 추출 없이 처리
# df_no_district = process_5th_governor_election6(
#     '부산시장개표진행상황읍면동별.xlsx',
#     header_rows=(3, 4),
#     filter_column='읍면동명',
#     filter_value='합계',
#     extract_district=False
# )

# 3. GitHub URL 사용
# github_url = "https://github.com/.../부산시장개표진행상황읍면동별.xlsx"
# df = process_5th_governor_election6(github_url)

## Seoul (3)


In [7]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_seoul = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C.xls'

seoul_6th = process_6th_governor_election3(
    file_path_or_url = blob_url6_seoul,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합계',
    add_sheet_name_column=True
)

발견된 시트: 25개
시트 이름: ['01종로구', '02중구', '03용산구', '04성동구', '05광진구', '06동대문구', '07중랑구', '08성북구', '09강북구', '10도봉구', '11노원구', '12은평구', '13서대문구', '14마포구', '15양천구', '16강서구', '17구로구', '18금천구', '19영등포구', '20동작구', '21관악구', '22서초구', '23강남구', '24송파구', '25강동구']


시트 1/25: '01종로구' 처리 중...
데이터 행 수: 55
필터링 적용: '읍면동명' == '합계' → 1행
'01종로구' 시트 처리 완료: 1행 추가됨

시트 2/25: '02중구' 처리 중...
데이터 행 수: 49
필터링 적용: '읍면동명' == '합계' → 1행
'02중구' 시트 처리 완료: 1행 추가됨

시트 3/25: '03용산구' 처리 중...
데이터 행 수: 52
필터링 적용: '읍면동명' == '합계' → 1행
'03용산구' 시트 처리 완료: 1행 추가됨

시트 4/25: '04성동구' 처리 중...
데이터 행 수: 55
필터링 적용: '읍면동명' == '합계' → 1행
'04성동구' 시트 처리 완료: 1행 추가됨

시트 5/25: '05광진구' 처리 중...
데이터 행 수: 49
필터링 적용: '읍면동명' == '합계' → 1행
'05광진구' 시트 처리 완료: 1행 추가됨

시트 6/25: '06동대문구' 처리 중...
데이터 행 수: 46
필터링 적용: '읍면동명' == '합계' → 1행
'06동대문구' 시트 처리 완료: 1행 추가됨

시트 7/25: '07중랑구' 처리 중...
데이터 행 수: 52
필터링 적용: '읍면동명' == '합계' → 1행
'07중랑구' 시트 처리 완료: 1행 추가됨

시트 8/25: '08성북구' 처리 중...
데이터 행 수: 64
필터링 적용: '읍면동명' == '합계' → 1행
'08성북구' 시트 처리 완료: 1행 추가됨

시트 9/25: '09강북구' 처리 중..

In [8]:
seoul_6th

,시트명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n정몽준,새정치민주연합\n박원순,통합진보당\n정태흥,새정치당\n홍정식,계,무효\n투표수,기권수
0,01종로구,합계,NaN,"136,925","81,113","34,599","45,020",330,238,"80,187",926,"55,812"
1,02중구,합계,NaN,"113,235","65,393","28,273","35,662",291,262,"64,488",905,"47,842"
2,03용산구,합계,NaN,"204,529","118,198","58,479","57,807",462,365,"117,113","1,085","86,331"
3,04성동구,합계,NaN,"252,308","146,873","62,376","81,645",715,546,"145,282","1,591","105,435"
4,05광진구,합계,NaN,"309,269","177,484","72,160","102,449",832,592,"176,033","1,451","131,785"
5,06동대문구,합계,NaN,"308,318","179,384","76,325","99,568",910,684,"177,487","1,897","128,934"
6,07중랑구,합계,NaN,"351,965","189,523","80,700","104,905","1,050",786,"187,441","2,082","162,442"
7,08성북구,합계,NaN,"394,293","229,201","92,897","131,919","1,187",844,"226,847","2,354","165,092"
8,09강북구,합계,NaN,"284,048","155,350","64,944","87,128",927,693,"153,692","1,658","128,698"
9,10도봉구,합계,NaN,"296,035","173,232","75,252","94,590",978,726,"171,546","1,686","122,803"


In [9]:
seoul_6th.columns.tolist()

['시트명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n정몽준',
 '새정치민주연합\n박원순',
 '통합진보당\n정태흥',
 '새정치당\n홍정식',
 '계',
 '무효\n투표수',
 '기권수']

In [10]:
rename_seoul = {
    '시트명': '구시군',
    '후보자별 득표수_새누리당\n정몽준': '득표수_1_새누리당_정몽준',
    '새정치민주연합\n박원순': '득표수_2_새정치민주연합_박원순',
    '통합진보당\n정태흥': '득표수_3_통합진보당_정태흥',
    '새정치당\n홍정식': '득표수_4_새정치국민의당_홍정식',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [11]:
seoul_6th = seoul_6th.rename(columns=rename_seoul).drop(columns=['읍면동명', '구분'])
seoul_6th

,구시군,선거인수,투표수,득표수_1_새누리당_정몽준,득표수_2_새정치민주연합_박원순,득표수_3_통합진보당_정태흥,득표수_4_새정치국민의당_홍정식,득표수_계,무효투표수,기권수
0,01종로구,"136,925","81,113","34,599","45,020",330,238,"80,187",926,"55,812"
1,02중구,"113,235","65,393","28,273","35,662",291,262,"64,488",905,"47,842"
2,03용산구,"204,529","118,198","58,479","57,807",462,365,"117,113","1,085","86,331"
3,04성동구,"252,308","146,873","62,376","81,645",715,546,"145,282","1,591","105,435"
4,05광진구,"309,269","177,484","72,160","102,449",832,592,"176,033","1,451","131,785"
5,06동대문구,"308,318","179,384","76,325","99,568",910,684,"177,487","1,897","128,934"
6,07중랑구,"351,965","189,523","80,700","104,905","1,050",786,"187,441","2,082","162,442"
7,08성북구,"394,293","229,201","92,897","131,919","1,187",844,"226,847","2,354","165,092"
8,09강북구,"284,048","155,350","64,944","87,128",927,693,"153,692","1,658","128,698"
9,10도봉구,"296,035","173,232","75,252","94,590",978,726,"171,546","1,686","122,803"


In [12]:
seoul_6th = seoul_6th.assign(
    시도='서울특별시',
    구시군=lambda df: df['구시군'].str.replace(r'^\d{2}', '', regex=True).str.strip()
)[['시도'] + seoul_6th.columns.tolist()]

In [13]:
seoul_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_정몽준,득표수_2_새정치민주연합_박원순,득표수_3_통합진보당_정태흥,득표수_4_새정치국민의당_홍정식,득표수_계,무효투표수,기권수
0,서울특별시,종로구,"136,925","81,113","34,599","45,020",330,238,"80,187",926,"55,812"
1,서울특별시,중구,"113,235","65,393","28,273","35,662",291,262,"64,488",905,"47,842"
2,서울특별시,용산구,"204,529","118,198","58,479","57,807",462,365,"117,113","1,085","86,331"
3,서울특별시,성동구,"252,308","146,873","62,376","81,645",715,546,"145,282","1,591","105,435"
4,서울특별시,광진구,"309,269","177,484","72,160","102,449",832,592,"176,033","1,451","131,785"


In [14]:
seoul_6th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 25 non-null     object
 1   구시군                25 non-null     object
 2   선거인수               25 non-null     object
 3   투표수                25 non-null     object
 4   득표수_1_새누리당_정몽준     25 non-null     object
 5   득표수_2_새정치민주연합_박원순  25 non-null     object
 6   득표수_3_통합진보당_정태흥    25 non-null     object
 7   득표수_4_새정치국민의당_홍정식  25 non-null     object
 8   득표수_계              25 non-null     object
 9   무효투표수              25 non-null     object
 10  기권수                25 non-null     object
dtypes: object(11)
memory usage: 2.3+ KB


In [15]:
seoul_6th = seoul_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [16]:
# 수치형 열만 합계 구하기
summary_row = seoul_6th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '서울특별시')

# summary_row를 맨 위에 붙이기
seoul_6th_with_total = pd.concat([summary_row, seoul_6th], ignore_index=True)

In [17]:
seoul_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_정몽준,득표수_2_새정치민주연합_박원순,득표수_3_통합진보당_정태흥,득표수_4_새정치국민의당_홍정식,득표수_계,무효투표수,기권수
0,서울특별시,합계,8441594,4948897,2109869,2752171,23638,17603,4903281,45616,3492697
1,서울특별시,종로구,136925,81113,34599,45020,330,238,80187,926,55812
2,서울특별시,중구,113235,65393,28273,35662,291,262,64488,905,47842
3,서울특별시,용산구,204529,118198,58479,57807,462,365,117113,1085,86331
4,서울특별시,성동구,252308,146873,62376,81645,715,546,145282,1591,105435
5,서울특별시,광진구,309269,177484,72160,102449,832,592,176033,1451,131785
6,서울특별시,동대문구,308318,179384,76325,99568,910,684,177487,1897,128934
7,서울특별시,중랑구,351965,189523,80700,104905,1050,786,187441,2082,162442
8,서울특별시,성북구,394293,229201,92897,131919,1187,844,226847,2354,165092
9,서울특별시,강북구,284048,155350,64944,87128,927,693,153692,1658,128698


In [18]:
seoul_6th_with_total.to_csv("temp1_governor_seoul_6.csv", index=False, encoding="utf-8-sig")


## Busan (6)


In [19]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_busan = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EB%B6%80%EC%82%B0-%EC%8B%9C%EC%9E%A5-%EA%B0%9C%ED%91%9C%EC%A7%84%ED%96%89%EC%83%81%ED%99%A9(%EC%9D%8D%EB%A9%B4%EB%8F%99%EB%B3%84).xlsx'

busan_6th = process_5th_governor_election6(
    file_path_or_url = blob_url6_busan,
    header_rows = (3, 4),  # 엑셀 기준 행 번호 (3행부터 4행까지)
    filter_column = '읍면동명',
    filter_value = '합계',
    extract_district = True
)

헤더 행: 3행 ~ 4행
생성된 컬럼 수: 9
데이터 행 수: 754

'읍면동명' 컬럼 발견: 읍면동명

구시군명 열 추가 완료

필터링 전: 754행 → 필터링 후: 16행
'읍면동명' == '합계'인 행만 추출

결과 데이터 샘플:
     구시군명 읍면동명   구분     선거인수      투표수 후보자별 득표수_새누리당\n서병수 무소속\n오거돈        계  \
0      중구   합계  NaN   41,868   23,404             12,576   10,057   22,633   
35     서구   합계  NaN  102,694   55,778             29,893   24,088   53,981   
82     동구   합계  NaN   83,350   47,730             25,288   20,693   45,981   
132   영도구   합계  NaN  115,944   60,321             30,436   27,462   57,898   
173  부산진구   합계  NaN  329,022  180,227             88,196   85,731  173,927   

    무효\n투표수      기권수  
0       771   18,464  
35    1,797   46,916  
82    1,749   35,620  
132   2,423   55,623  
173   6,300  148,795  


In [20]:
busan_6th

,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n서병수,무소속\n오거돈,계,무효\n투표수,기권수
0,중구,합계,NaN,"41,868","23,404","12,576","10,057","22,633",771,"18,464"
35,서구,합계,NaN,"102,694","55,778","29,893","24,088","53,981","1,797","46,916"
82,동구,합계,NaN,"83,350","47,730","25,288","20,693","45,981","1,749","35,620"
132,영도구,합계,NaN,"115,944","60,321","30,436","27,462","57,898","2,423","55,623"
173,부산진구,합계,NaN,"329,022","180,227","88,196","85,731","173,927","6,300","148,795"
250,동래구,합계,NaN,"229,691","130,592","64,409","62,544","126,953","3,639","99,099"
297,남구,합계,NaN,"241,177","137,349","66,903","66,081","132,984","4,365","103,828"
356,북구,합계,NaN,"251,760","139,860","65,060","69,774","134,834","5,026","111,900"
403,해운대구,합계,NaN,"342,080","188,267","91,704","91,013","182,717","5,550","153,813"
465,기장군,합계,NaN,"107,360","62,794","29,611","30,625","60,236","2,558","44,566"


In [21]:
busan_6th.columns.tolist()

['구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n서병수',
 '무소속\n오거돈',
 '계',
 '무효\n투표수',
 '기권수']

In [22]:
rename_busan = {
    '구시군명': '구시군',
    '후보자별 득표수_새누리당\n서병수': '득표수_1_새누리당_서병수',
    '무소속\n오거돈': '득표수_4_무소속_오거돈',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [23]:
busan_6th = busan_6th.rename(columns=rename_busan).drop(columns=['읍면동명', '구분'])
busan_6th

,구시군,선거인수,투표수,득표수_1_새누리당_서병수,득표수_4_무소속_오거돈,득표수_계,무효투표수,기권수
0,중구,"41,868","23,404","12,576","10,057","22,633",771,"18,464"
35,서구,"102,694","55,778","29,893","24,088","53,981","1,797","46,916"
82,동구,"83,350","47,730","25,288","20,693","45,981","1,749","35,620"
132,영도구,"115,944","60,321","30,436","27,462","57,898","2,423","55,623"
173,부산진구,"329,022","180,227","88,196","85,731","173,927","6,300","148,795"
250,동래구,"229,691","130,592","64,409","62,544","126,953","3,639","99,099"
297,남구,"241,177","137,349","66,903","66,081","132,984","4,365","103,828"
356,북구,"251,760","139,860","65,060","69,774","134,834","5,026","111,900"
403,해운대구,"342,080","188,267","91,704","91,013","182,717","5,550","153,813"
465,기장군,"107,360","62,794","29,611","30,625","60,236","2,558","44,566"


In [24]:
busan_6th = busan_6th.assign(
    시도='부산광역시'
)[['시도'] + busan_6th.columns.tolist()]

In [25]:
busan_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_서병수,득표수_4_무소속_오거돈,득표수_계,무효투표수,기권수
0,부산광역시,중구,"41,868","23,404","12,576","10,057","22,633",771,"18,464"
35,부산광역시,서구,"102,694","55,778","29,893","24,088","53,981","1,797","46,916"
82,부산광역시,동구,"83,350","47,730","25,288","20,693","45,981","1,749","35,620"
132,부산광역시,영도구,"115,944","60,321","30,436","27,462","57,898","2,423","55,623"
173,부산광역시,부산진구,"329,022","180,227","88,196","85,731","173,927","6,300","148,795"


In [26]:
busan_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 714
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   시도              16 non-null     object
 1   구시군             16 non-null     object
 2   선거인수            16 non-null     object
 3   투표수             16 non-null     object
 4   득표수_1_새누리당_서병수  16 non-null     object
 5   득표수_4_무소속_오거돈   16 non-null     object
 6   득표수_계           16 non-null     object
 7   무효투표수           16 non-null     object
 8   기권수             16 non-null     object
dtypes: object(9)
memory usage: 1.8+ KB


In [27]:
busan_6th = busan_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [28]:
# 수치형 열만 합계 구하기
summary_row = busan_6th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '부산광역시')

# summary_row를 맨 위에 붙이기
busan_6th_with_total = pd.concat([summary_row, busan_6th], ignore_index=True)

In [29]:
busan_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_서병수,득표수_4_무소속_오거돈,득표수_계,무효투표수,기권수
0,부산광역시,합계,2932179,1629167,797926,777225,1575151,54016,1303012
1,부산광역시,중구,41868,23404,12576,10057,22633,771,18464
2,부산광역시,서구,102694,55778,29893,24088,53981,1797,46916
3,부산광역시,동구,83350,47730,25288,20693,45981,1749,35620
4,부산광역시,영도구,115944,60321,30436,27462,57898,2423,55623
5,부산광역시,부산진구,329022,180227,88196,85731,173927,6300,148795
6,부산광역시,동래구,229691,130592,64409,62544,126953,3639,99099
7,부산광역시,남구,241177,137349,66903,66081,132984,4365,103828
8,부산광역시,북구,251760,139860,65060,69774,134834,5026,111900
9,부산광역시,해운대구,342080,188267,91704,91013,182717,5550,153813


In [30]:
busan_6th_with_total.to_csv("temp1_governor_busan_6.csv", index=False, encoding="utf-8-sig")


## Daegu (4)


In [31]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_daegu = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EB%8C%80%EA%B5%AC%EA%B4%91%EC%97%AD%EC%8B%9C.xls'

daegu_6th = process_6th_governor_election4(
    file_path_or_url = blob_url6_daegu,
    header_rows = (3, 4),
    filter_column = None,
    filter_value = None,
    row_range=(1,13)  # 읽을 행 범위 (엑셀 기준)
)

파일에서 1행부터 13행까지만 읽습니다.
헤더 행: 3행 ~ 4행
생성된 컬럼 수: 12
데이터 행 수: 9

생성된 컬럼명 (처음 10개):
0: 대구광역시
1: 구분
2: 선거인수
3: 투표수
4: 후보자별 득표수_새누리당
권영진
5: 새정치민주연합
김부겸
6: 통합진보당
송영우
7: 정의당
이원준
8: 무소속
이정숙
9: 계


In [32]:
daegu_6th

,대구광역시,구분,선거인수,투표수,후보자별 득표수_새누리당\n권영진,새정치민주연합\n김부겸,통합진보당\n송영우,정의당\n이원준,무소속\n이정숙,계,무효\n투표수,기권수
0,합계,NaN,"2,012,579","1,052,638","581,175\n(55.95)","418,891\n(40.33)","10,857\n(1.04)","12,922\n(1.24)","14,774\n(1.42)","1,038,619","14,019","959,941"
1,중구,NaN,"65,996","35,719","20,419\n(58.18)","13,410\n(38.21)",313\n(0.89),464\n(1.32),485\n(1.38),"35,091",628,"30,277"
2,동구,NaN,"286,747","151,471","86,217\n(57.80)","56,920\n(38.16)","2,065\n(1.38)","1,724\n(1.15)","2,214\n(1.48)","149,140","2,331","135,276"
3,서구,NaN,"183,116","91,230","56,272\n(62.85)","29,458\n(32.90)",981\n(1.09),"1,209\n(1.35)","1,609\n(1.79)","89,529","1,701","91,886"
4,남구,NaN,"142,345","70,076","42,488\n(61.55)","24,071\n(34.87)",631\n(0.91),817\n(1.18),"1,015\n(1.47)","69,022","1,054","72,269"
5,북구,NaN,"348,883","179,341","99,100\n(55.97)","70,646\n(39.90)","2,189\n(1.23)","2,359\n(1.33)","2,758\n(1.55)","177,052","2,289","169,542"
6,수성구,NaN,"359,016","201,364","99,577\n(49.93)","94,715\n(47.49)","1,304\n(0.65)","1,886\n(0.94)","1,950\n(0.97)","199,432","1,932","157,652"
7,달서구,NaN,"481,475","249,724","134,637\n(54.54)","103,302\n(41.84)","2,324\n(0.94)","3,354\n(1.35)","3,230\n(1.30)","246,847","2,877","231,751"
8,달성군,NaN,"145,001","73,713","42,465\n(58.56)","26,369\n(36.36)","1,050\n(1.44)","1,109\n(1.52)","1,513\n(2.08)","72,506","1,207","71,288"


In [33]:
daegu_6th.columns.tolist()

['대구광역시',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n권영진',
 '새정치민주연합\n김부겸',
 '통합진보당\n송영우',
 '정의당\n이원준',
 '무소속\n이정숙',
 '계',
 '무효\n투표수',
 '기권수']

In [34]:
rename_daegu = {
    '대구광역시': '구시군',
    '후보자별 득표수_새누리당\n권영진': '득표수_1_새누리당_권영진',
    '새정치민주연합\n김부겸': '득표수_2_새정치민주연합_김부겸',
    '통합진보당\n송영우': '득표수_3_통합진보당_송영우',
    '정의당\n이원준': '득표수_4_정의당_이원준',
    '무소속\n이정숙': '득표수_5_무소속_이정숙',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [35]:
daegu_6th = daegu_6th.rename(columns=rename_daegu).drop(columns=['구분'])
daegu_6th

,구시군,선거인수,투표수,득표수_1_새누리당_권영진,득표수_2_새정치민주연합_김부겸,득표수_3_통합진보당_송영우,득표수_4_정의당_이원준,득표수_5_무소속_이정숙,득표수_계,무효투표수,기권수
0,합계,"2,012,579","1,052,638","581,175\n(55.95)","418,891\n(40.33)","10,857\n(1.04)","12,922\n(1.24)","14,774\n(1.42)","1,038,619","14,019","959,941"
1,중구,"65,996","35,719","20,419\n(58.18)","13,410\n(38.21)",313\n(0.89),464\n(1.32),485\n(1.38),"35,091",628,"30,277"
2,동구,"286,747","151,471","86,217\n(57.80)","56,920\n(38.16)","2,065\n(1.38)","1,724\n(1.15)","2,214\n(1.48)","149,140","2,331","135,276"
3,서구,"183,116","91,230","56,272\n(62.85)","29,458\n(32.90)",981\n(1.09),"1,209\n(1.35)","1,609\n(1.79)","89,529","1,701","91,886"
4,남구,"142,345","70,076","42,488\n(61.55)","24,071\n(34.87)",631\n(0.91),817\n(1.18),"1,015\n(1.47)","69,022","1,054","72,269"
5,북구,"348,883","179,341","99,100\n(55.97)","70,646\n(39.90)","2,189\n(1.23)","2,359\n(1.33)","2,758\n(1.55)","177,052","2,289","169,542"
6,수성구,"359,016","201,364","99,577\n(49.93)","94,715\n(47.49)","1,304\n(0.65)","1,886\n(0.94)","1,950\n(0.97)","199,432","1,932","157,652"
7,달서구,"481,475","249,724","134,637\n(54.54)","103,302\n(41.84)","2,324\n(0.94)","3,354\n(1.35)","3,230\n(1.30)","246,847","2,877","231,751"
8,달성군,"145,001","73,713","42,465\n(58.56)","26,369\n(36.36)","1,050\n(1.44)","1,109\n(1.52)","1,513\n(2.08)","72,506","1,207","71,288"


In [36]:
daegu_6th = daegu_6th.assign(
    시도='대구광역시'
)[['시도'] + daegu_6th.columns.tolist()]

In [37]:
daegu_6th.head(3)

,시도,구시군,선거인수,투표수,득표수_1_새누리당_권영진,득표수_2_새정치민주연합_김부겸,득표수_3_통합진보당_송영우,득표수_4_정의당_이원준,득표수_5_무소속_이정숙,득표수_계,무효투표수,기권수
0,대구광역시,합계,"2,012,579","1,052,638","581,175\n(55.95)","418,891\n(40.33)","10,857\n(1.04)","12,922\n(1.24)","14,774\n(1.42)","1,038,619","14,019","959,941"
1,대구광역시,중구,"65,996","35,719","20,419\n(58.18)","13,410\n(38.21)",313\n(0.89),464\n(1.32),485\n(1.38),"35,091",628,"30,277"
2,대구광역시,동구,"286,747","151,471","86,217\n(57.80)","56,920\n(38.16)","2,065\n(1.38)","1,724\n(1.15)","2,214\n(1.48)","149,140","2,331","135,276"


In [38]:
daegu_6th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 9 non-null      object
 1   구시군                9 non-null      object
 2   선거인수               9 non-null      object
 3   투표수                9 non-null      object
 4   득표수_1_새누리당_권영진     9 non-null      object
 5   득표수_2_새정치민주연합_김부겸  9 non-null      object
 6   득표수_3_통합진보당_송영우    9 non-null      object
 7   득표수_4_정의당_이원준      9 non-null      object
 8   득표수_5_무소속_이정숙      9 non-null      object
 9   득표수_계              9 non-null      object
 10  무효투표수              9 non-null      object
 11  기권수                9 non-null      object
dtypes: object(12)
memory usage: 996.0+ bytes


In [39]:
daegu_6th = daegu_6th.apply(
    lambda col: col.str.replace(r'\n.*$', '', regex=True)
    if col.dtype == 'object' and col.astype(str).str.contains('\n').any()
    else col
).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [40]:
daegu_6th_with_total = daegu_6th

In [41]:
daegu_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_권영진,득표수_2_새정치민주연합_김부겸,득표수_3_통합진보당_송영우,득표수_4_정의당_이원준,득표수_5_무소속_이정숙,득표수_계,무효투표수,기권수
0,대구광역시,합계,2012579,1052638,581175,418891,10857,12922,14774,1038619,14019,959941
1,대구광역시,중구,65996,35719,20419,13410,313,464,485,35091,628,30277
2,대구광역시,동구,286747,151471,86217,56920,2065,1724,2214,149140,2331,135276
3,대구광역시,서구,183116,91230,56272,29458,981,1209,1609,89529,1701,91886
4,대구광역시,남구,142345,70076,42488,24071,631,817,1015,69022,1054,72269
5,대구광역시,북구,348883,179341,99100,70646,2189,2359,2758,177052,2289,169542
6,대구광역시,수성구,359016,201364,99577,94715,1304,1886,1950,199432,1932,157652
7,대구광역시,달서구,481475,249724,134637,103302,2324,3354,3230,246847,2877,231751
8,대구광역시,달성군,145001,73713,42465,26369,1050,1109,1513,72506,1207,71288


In [42]:
daegu_6th_with_total.to_csv("temp1_governor_daegu_6.csv", index=False, encoding="utf-8-sig")


## Incheon (1)


In [43]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_incheon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%9D%B8%EC%B2%9C%EA%B4%91%EC%97%AD%EC%8B%9C.xls'

incheon_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_incheon,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합계'
)

헤더 행: 3행 ~ 4행
생성된 컬럼 수: 11
데이터 행 수: 482

생성된 컬럼명 (처음 10개):
0: 구시군명
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
유정복
6: 새정치민주연합
송영길
7: 통합진보당
신창현
8: 계
9: 무효
투표수

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 482행 → 필터링 후: 11행
'읍면동명' == '합계'인 행만 추출


In [44]:
incheon_6th

,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n유정복,새정치민주연합\n송영길,통합진보당\n신창현,계,무효\n투표수,기권수
0,인천광역시,합계,NaN,"2,319,198","1,244,502","615,077","593,555","22,651","1,231,283","13,219","1,074,696"
1,중구,합계,NaN,"89,767","48,565","26,324","20,788",845,"47,957",608,"41,202"
38,동구,합계,NaN,"61,693","36,534","19,573","15,605",711,"35,889",645,"25,159"
75,남구,합계,NaN,"340,427","174,258","92,490","76,984","2,854","172,328","1,930","166,169"
142,연수구,합계,NaN,"238,453","136,706","71,942","61,389","2,154","135,485","1,221","101,747"
182,남동구,합계,NaN,"405,404","215,359","102,688","106,433","4,308","213,429","1,930","190,045"
243,부평구,합계,NaN,"451,971","237,759","108,912","121,970","4,670","235,552","2,207","214,212"
313,계양구,합계,NaN,"274,659","143,387","61,275","78,343","2,562","142,180","1,207","131,272"
350,서구,합계,NaN,"380,611","199,733","98,389","96,162","3,413","197,964","1,769","180,878"
414,강화군,합계,NaN,"58,089","38,611","24,762","11,854",815,"37,431","1,180","19,478"


In [45]:
incheon_6th.columns.tolist()

['구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n유정복',
 '새정치민주연합\n송영길',
 '통합진보당\n신창현',
 '계',
 '무효\n투표수',
 '기권수']

In [46]:
rename_incheon = {
    '구시군명': '구시군',
    '후보자별 득표수_새누리당\n유정복': '득표수_1_새누리당_유정복',
    '새정치민주연합\n송영길': '득표수_2_새정치민주연합_송영길',
    '통합진보당\n신창현': '득표수_3_통합진보당_신창현',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [47]:
incheon_6th = incheon_6th.rename(columns=rename_incheon).drop(columns=['읍면동명', '구분'])
incheon_6th

,구시군,선거인수,투표수,득표수_1_새누리당_유정복,득표수_2_새정치민주연합_송영길,득표수_3_통합진보당_신창현,득표수_계,무효투표수,기권수
0,인천광역시,"2,319,198","1,244,502","615,077","593,555","22,651","1,231,283","13,219","1,074,696"
1,중구,"89,767","48,565","26,324","20,788",845,"47,957",608,"41,202"
38,동구,"61,693","36,534","19,573","15,605",711,"35,889",645,"25,159"
75,남구,"340,427","174,258","92,490","76,984","2,854","172,328","1,930","166,169"
142,연수구,"238,453","136,706","71,942","61,389","2,154","135,485","1,221","101,747"
182,남동구,"405,404","215,359","102,688","106,433","4,308","213,429","1,930","190,045"
243,부평구,"451,971","237,759","108,912","121,970","4,670","235,552","2,207","214,212"
313,계양구,"274,659","143,387","61,275","78,343","2,562","142,180","1,207","131,272"
350,서구,"380,611","199,733","98,389","96,162","3,413","197,964","1,769","180,878"
414,강화군,"58,089","38,611","24,762","11,854",815,"37,431","1,180","19,478"


In [48]:
incheon_6th = incheon_6th.assign(
    시도='인천광역시'
)[['시도'] + incheon_6th.columns.tolist()]

In [49]:
incheon_6th.loc[0, '구시군'] = '합계'

In [50]:
incheon_6th.head(3)

,시도,구시군,선거인수,투표수,득표수_1_새누리당_유정복,득표수_2_새정치민주연합_송영길,득표수_3_통합진보당_신창현,득표수_계,무효투표수,기권수
0,인천광역시,합계,"2,319,198","1,244,502","615,077","593,555","22,651","1,231,283","13,219","1,074,696"
1,인천광역시,중구,"89,767","48,565","26,324","20,788",845,"47,957",608,"41,202"
38,인천광역시,동구,"61,693","36,534","19,573","15,605",711,"35,889",645,"25,159"


In [51]:
incheon_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 0 to 457
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 11 non-null     object
 1   구시군                11 non-null     object
 2   선거인수               11 non-null     object
 3   투표수                11 non-null     object
 4   득표수_1_새누리당_유정복     11 non-null     object
 5   득표수_2_새정치민주연합_송영길  11 non-null     object
 6   득표수_3_통합진보당_신창현    11 non-null     object
 7   득표수_계              11 non-null     object
 8   무효투표수              11 non-null     object
 9   기권수                11 non-null     object
dtypes: object(10)
memory usage: 1.2+ KB


In [52]:
incheon_6th = incheon_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [53]:
incheon_6th_with_total = incheon_6th

In [54]:
incheon_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_유정복,득표수_2_새정치민주연합_송영길,득표수_3_통합진보당_신창현,득표수_계,무효투표수,기권수
0,인천광역시,합계,2319198,1244502,615077,593555,22651,1231283,13219,1074696
1,인천광역시,중구,89767,48565,26324,20788,845,47957,608,41202
38,인천광역시,동구,61693,36534,19573,15605,711,35889,645,25159
75,인천광역시,남구,340427,174258,92490,76984,2854,172328,1930,166169
142,인천광역시,연수구,238453,136706,71942,61389,2154,135485,1221,101747
182,인천광역시,남동구,405404,215359,102688,106433,4308,213429,1930,190045
243,인천광역시,부평구,451971,237759,108912,121970,4670,235552,2207,214212
313,인천광역시,계양구,274659,143387,61275,78343,2562,142180,1207,131272
350,인천광역시,서구,380611,199733,98389,96162,3413,197964,1769,180878
414,인천광역시,강화군,58089,38611,24762,11854,815,37431,1180,19478


In [55]:
incheon_6th_with_total.to_csv("temp1_governor_incheon_6.csv", index=False, encoding="utf-8-sig")


## Gwangju (3)


In [56]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_gwangju = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EA%B4%91%EC%A3%BC%EA%B4%91%EC%97%AD%EC%8B%9C.xlsx'

gwangju_6th = process_6th_governor_election3(
    file_path_or_url = blob_url6_gwangju,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합계',
    add_sheet_name_column=True
)

발견된 시트: 5개
시트 이름: ['동구', '서구', '남구', '북구', '광산구']


시트 1/5: '동구' 처리 중...
데이터 행 수: 43
필터링 적용: '읍면동명' == '합계' → 1행
'동구' 시트 처리 완료: 1행 추가됨

시트 2/5: '서구' 처리 중...
데이터 행 수: 58
필터링 적용: '읍면동명' == '합계' → 1행
'서구' 시트 처리 완료: 1행 추가됨

시트 3/5: '남구' 처리 중...
데이터 행 수: 52
필터링 적용: '읍면동명' == '합계' → 1행
'남구' 시트 처리 완료: 1행 추가됨

시트 4/5: '북구' 처리 중...
데이터 행 수: 87
필터링 적용: '읍면동명' == '합계' → 1행
'북구' 시트 처리 완료: 1행 추가됨

시트 5/5: '광산구' 처리 중...
데이터 행 수: 67
필터링 적용: '읍면동명' == '합계' → 1행
'광산구' 시트 처리 완료: 1행 추가됨

최종 결과: 총 5행의 데이터


In [57]:
gwangju_6th

,시트명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n이정재,새정치민주연합\n윤장현,통합진보당\n윤민호,노동당\n이병훈,무소속\n강운태,무소속\n이병완,계,무효\n투표수,기권수
0,동구,합계,NaN,"86,465","51,253","1,889","28,605","1,238",591,"16,551","1,091","49,965","1,288","35,212"
1,서구,합계,NaN,"242,201","142,219","4,481","82,754","3,700","1,225","43,444","3,932","139,536","2,683","99,982"
2,남구,합계,NaN,171997,101811,3160,57010,2357,816,33798,2389,99530,2281,70186
3,북구,합계,NaN,"351,880","198,547","6,840","113,138","6,912","2,201","59,878","4,995","193,964","4,583","153,333"
4,광산구,합계,NaN,"285,875","156,178","5,244","85,696","6,993","1,952","47,995","3,842","151,722","4,456","129,697"


In [58]:
gwangju_6th.columns.tolist()

['시트명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n이정재',
 '새정치민주연합\n윤장현',
 '통합진보당\n윤민호',
 '노동당\n이병훈',
 '무소속\n강운태',
 '무소속\n이병완',
 '계',
 '무효\n투표수',
 '기권수']

In [59]:
rename_gwangju = {
    '시트명': '구시군',
    '후보자별 득표수_새누리당\n이정재': '득표수_1_새누리당_이정재',
    '새정치민주연합\n윤장현': '득표수_2_새정치민주연합_윤장현',
    '통합진보당\n윤민호': '득표수_3_통합진보당_윤민호',
    '노동당\n이병훈': '득표수_4_노동당_이병훈',
    '무소속\n강운태': '득표수_5_무소속_강운태',
    '무소속\n이병완': '득표수_5_무소속_이병완',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [60]:
gwangju_6th = gwangju_6th.rename(columns=rename_gwangju).drop(columns=['읍면동명', '구분'])
gwangju_6th

,구시군,선거인수,투표수,득표수_1_새누리당_이정재,득표수_2_새정치민주연합_윤장현,득표수_3_통합진보당_윤민호,득표수_4_노동당_이병훈,득표수_5_무소속_강운태,득표수_5_무소속_이병완,득표수_계,무효투표수,기권수
0,동구,"86,465","51,253","1,889","28,605","1,238",591,"16,551","1,091","49,965","1,288","35,212"
1,서구,"242,201","142,219","4,481","82,754","3,700","1,225","43,444","3,932","139,536","2,683","99,982"
2,남구,171997,101811,3160,57010,2357,816,33798,2389,99530,2281,70186
3,북구,"351,880","198,547","6,840","113,138","6,912","2,201","59,878","4,995","193,964","4,583","153,333"
4,광산구,"285,875","156,178","5,244","85,696","6,993","1,952","47,995","3,842","151,722","4,456","129,697"


In [61]:
gwangju_6th = gwangju_6th.assign(
    시도='광주광역시'
)[['시도'] + gwangju_6th.columns.tolist()]

In [62]:
gwangju_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_이정재,득표수_2_새정치민주연합_윤장현,득표수_3_통합진보당_윤민호,득표수_4_노동당_이병훈,득표수_5_무소속_강운태,득표수_5_무소속_이병완,득표수_계,무효투표수,기권수
0,광주광역시,동구,"86,465","51,253","1,889","28,605","1,238",591,"16,551","1,091","49,965","1,288","35,212"
1,광주광역시,서구,"242,201","142,219","4,481","82,754","3,700","1,225","43,444","3,932","139,536","2,683","99,982"
2,광주광역시,남구,171997,101811,3160,57010,2357,816,33798,2389,99530,2281,70186
3,광주광역시,북구,"351,880","198,547","6,840","113,138","6,912","2,201","59,878","4,995","193,964","4,583","153,333"
4,광주광역시,광산구,"285,875","156,178","5,244","85,696","6,993","1,952","47,995","3,842","151,722","4,456","129,697"


In [63]:
gwangju_6th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 5 non-null      object
 1   구시군                5 non-null      object
 2   선거인수               5 non-null      object
 3   투표수                5 non-null      object
 4   득표수_1_새누리당_이정재     5 non-null      object
 5   득표수_2_새정치민주연합_윤장현  5 non-null      object
 6   득표수_3_통합진보당_윤민호    5 non-null      object
 7   득표수_4_노동당_이병훈      5 non-null      object
 8   득표수_5_무소속_강운태      5 non-null      object
 9   득표수_5_무소속_이병완      5 non-null      object
 10  득표수_계              5 non-null      object
 11  무효투표수              5 non-null      object
 12  기권수                5 non-null      object
dtypes: object(13)
memory usage: 652.0+ bytes


In [64]:
gwangju_6th = gwangju_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [65]:
# 수치형 열만 합계 구하기
summary_row = gwangju_6th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '광주광역시')

# summary_row를 맨 위에 붙이기
gwangju_6th_with_total = pd.concat([summary_row, gwangju_6th], ignore_index=True)

In [66]:
gwangju_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_이정재,득표수_2_새정치민주연합_윤장현,득표수_3_통합진보당_윤민호,득표수_4_노동당_이병훈,득표수_5_무소속_강운태,득표수_5_무소속_이병완,득표수_계,무효투표수,기권수
0,광주광역시,합계,1138418,650008,21614,367203,21200,6785,201666,16249,634717,15291,488410
1,광주광역시,동구,86465,51253,1889,28605,1238,591,16551,1091,49965,1288,35212
2,광주광역시,서구,242201,142219,4481,82754,3700,1225,43444,3932,139536,2683,99982
3,광주광역시,남구,171997,101811,3160,57010,2357,816,33798,2389,99530,2281,70186
4,광주광역시,북구,351880,198547,6840,113138,6912,2201,59878,4995,193964,4583,153333
5,광주광역시,광산구,285875,156178,5244,85696,6993,1952,47995,3842,151722,4456,129697


In [67]:
gwangju_6th_with_total.to_csv("temp1_governor_gwangju_6.csv", index=False, encoding="utf-8-sig")


## Daejeon (3)


In [68]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_daejeon = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EB%8C%80%EC%A0%84%EA%B4%91%EC%97%AD%EC%8B%9C.xls'

daejeon_6th = process_6th_governor_election3(
    file_path_or_url = blob_url6_daejeon,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합계',
    add_sheet_name_column=True
)

발견된 시트: 5개
시트 이름: ['동구', '중구', '서구', '유성구', '대덕구']


시트 1/5: '동구' 처리 중...
데이터 행 수: 52
필터링 적용: '읍면동명' == '합계' → 1행
'동구' 시트 처리 완료: 1행 추가됨

시트 2/5: '중구' 처리 중...
데이터 행 수: 55
필터링 적용: '읍면동명' == '합계' → 1행
'중구' 시트 처리 완료: 1행 추가됨

시트 3/5: '서구' 처리 중...
데이터 행 수: 73
필터링 적용: '읍면동명' == '합계' → 1행
'서구' 시트 처리 완료: 1행 추가됨

시트 4/5: '유성구' 처리 중...
데이터 행 수: 34
필터링 적용: '읍면동명' == '합계' → 1행
'유성구' 시트 처리 완료: 1행 추가됨

시트 5/5: '대덕구' 처리 중...
데이터 행 수: 40
필터링 적용: '읍면동명' == '합계' → 1행
'대덕구' 시트 처리 완료: 1행 추가됨

최종 결과: 총 5행의 데이터


In [69]:
daejeon_6th

,시트명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n박성효,새정치민주연합\n권선택,통합진보당\n김창근,정의당\n한창민,계,무효\n투표수,기권수
0,동구,합계,NaN,"202,671","104,754","51,063","48,607","1,422","1,913","103,005","1,749","97,917"
1,중구,합계,NaN,"213,239","116,220","55,956","55,596","1,209","1,754","114,515","1,705","97,019"
2,서구,합계,NaN,"389,886","209,058","96,154","104,428","2,608","3,648","206,838","2,220","180,828"
3,유성구,합계,NaN,"240,489","138,723","57,099","75,806","1,971","2,511","137,387","1,336","101,766"
4,대덕구,합계,NaN,"161,687","83,916","41,117","38,325","1,799","1,520","82,761","1,155","77,771"


In [70]:
daejeon_6th.columns.tolist()

['시트명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n박성효',
 '새정치민주연합\n권선택',
 '통합진보당\n김창근',
 '정의당\n한창민',
 '계',
 '무효\n투표수',
 '기권수']

In [71]:
rename_daejeon = {
    '시트명': '구시군',
    '후보자별 득표수_새누리당\n박성효': '득표수_1_새누리당_박성효',
    '새정치민주연합\n권선택': '득표수_2_새정치민주연합_권선택',
    '통합진보당\n김창근': '득표수_3_통합진보당_김창근',
    '정의당\n한창민': '득표수_4_정의당_한창민',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [72]:
daejeon_6th = daejeon_6th.rename(columns=rename_daejeon).drop(columns=['읍면동명', '구분'])
daejeon_6th

,구시군,선거인수,투표수,득표수_1_새누리당_박성효,득표수_2_새정치민주연합_권선택,득표수_3_통합진보당_김창근,득표수_4_정의당_한창민,득표수_계,무효투표수,기권수
0,동구,"202,671","104,754","51,063","48,607","1,422","1,913","103,005","1,749","97,917"
1,중구,"213,239","116,220","55,956","55,596","1,209","1,754","114,515","1,705","97,019"
2,서구,"389,886","209,058","96,154","104,428","2,608","3,648","206,838","2,220","180,828"
3,유성구,"240,489","138,723","57,099","75,806","1,971","2,511","137,387","1,336","101,766"
4,대덕구,"161,687","83,916","41,117","38,325","1,799","1,520","82,761","1,155","77,771"


In [73]:
daejeon_6th = daejeon_6th.assign(
    시도='대전광역시'
)[['시도'] + daejeon_6th.columns.tolist()]

In [74]:
daejeon_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박성효,득표수_2_새정치민주연합_권선택,득표수_3_통합진보당_김창근,득표수_4_정의당_한창민,득표수_계,무효투표수,기권수
0,대전광역시,동구,"202,671","104,754","51,063","48,607","1,422","1,913","103,005","1,749","97,917"
1,대전광역시,중구,"213,239","116,220","55,956","55,596","1,209","1,754","114,515","1,705","97,019"
2,대전광역시,서구,"389,886","209,058","96,154","104,428","2,608","3,648","206,838","2,220","180,828"
3,대전광역시,유성구,"240,489","138,723","57,099","75,806","1,971","2,511","137,387","1,336","101,766"
4,대전광역시,대덕구,"161,687","83,916","41,117","38,325","1,799","1,520","82,761","1,155","77,771"


In [75]:
daejeon_6th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 5 non-null      object
 1   구시군                5 non-null      object
 2   선거인수               5 non-null      object
 3   투표수                5 non-null      object
 4   득표수_1_새누리당_박성효     5 non-null      object
 5   득표수_2_새정치민주연합_권선택  5 non-null      object
 6   득표수_3_통합진보당_김창근    5 non-null      object
 7   득표수_4_정의당_한창민      5 non-null      object
 8   득표수_계              5 non-null      object
 9   무효투표수              5 non-null      object
 10  기권수                5 non-null      object
dtypes: object(11)
memory usage: 572.0+ bytes


In [76]:
daejeon_6th = daejeon_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [77]:
# 수치형 열만 합계 구하기
summary_row = daejeon_6th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '대전광역시')

# summary_row를 맨 위에 붙이기
daejeon_6th_with_total = pd.concat([summary_row, daejeon_6th], ignore_index=True)

In [78]:
daejeon_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박성효,득표수_2_새정치민주연합_권선택,득표수_3_통합진보당_김창근,득표수_4_정의당_한창민,득표수_계,무효투표수,기권수
0,대전광역시,합계,1207972,652671,301389,322762,9009,11346,644506,8165,555301
1,대전광역시,동구,202671,104754,51063,48607,1422,1913,103005,1749,97917
2,대전광역시,중구,213239,116220,55956,55596,1209,1754,114515,1705,97019
3,대전광역시,서구,389886,209058,96154,104428,2608,3648,206838,2220,180828
4,대전광역시,유성구,240489,138723,57099,75806,1971,2511,137387,1336,101766
5,대전광역시,대덕구,161687,83916,41117,38325,1799,1520,82761,1155,77771


In [79]:
daejeon_6th_with_total.to_csv("temp1_governor_daejeon_6.csv", index=False, encoding="utf-8-sig")


## Ulsan (1)


In [80]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_ulsan = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%9A%B8%EC%82%B0%EA%B4%91%EC%97%AD%EC%8B%9C.xls'

ulsan_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_ulsan,
    header_rows = (3, 4),
    filter_column = '구분',
    filter_value = '합계'
)

헤더 행: 3행 ~ 4행
생성된 컬럼 수: 11
데이터 행 수: 189

생성된 컬럼명 (처음 10개):
0: 구시군명
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
김기현
6: 정의당
조승수
7: 노동당
이갑용
8: 계
9: 무효
투표수

'구분' 컬럼 발견: 구분
필터링 전: 189행 → 필터링 후: 6행
'구분' == '합계'인 행만 추출


In [81]:
ulsan_6th

,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n김기현,정의당\n조승수,노동당\n이갑용,계,무효\n투표수,기권수
0,울산광역시,NaN,합계,"912,325","511,881","306,311","123,736","38,107","468,154","43,727","400,444"
1,중구,NaN,합계,"189,218","106,699","66,129","24,913","6,912","97,954","8,745","82,519"
44,남구,NaN,합계,"277,480","151,643","95,192","36,887","8,032","140,111","11,532","125,837"
90,동구,NaN,합계,"141,266","81,696","44,961","17,794","10,811","73,566","8,130","59,570"
121,북구,NaN,합계,"139,347","79,777","42,353","24,380","6,698","73,431","6,346","59,570"
149,울주군,NaN,합계,"165,014","92,066","57,676","19,762","5,654","83,092","8,974","72,948"


In [82]:
ulsan_6th.columns.tolist()

['구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n김기현',
 '정의당\n조승수',
 '노동당\n이갑용',
 '계',
 '무효\n투표수',
 '기권수']

In [83]:
rename_ulsan = {
    '구시군명': '구시군',
    '후보자별 득표수_새누리당\n김기현': '득표수_1_새누리당_김기현',
    '정의당\n조승수': '득표수_4_정의당_조승수',
    '노동당\n이갑용': '득표수_5_노동당_이갑용',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [84]:
ulsan_6th = ulsan_6th.rename(columns=rename_ulsan).drop(columns=['읍면동명', '구분'])
ulsan_6th

,구시군,선거인수,투표수,득표수_1_새누리당_김기현,득표수_4_정의당_조승수,득표수_5_노동당_이갑용,득표수_계,무효투표수,기권수
0,울산광역시,"912,325","511,881","306,311","123,736","38,107","468,154","43,727","400,444"
1,중구,"189,218","106,699","66,129","24,913","6,912","97,954","8,745","82,519"
44,남구,"277,480","151,643","95,192","36,887","8,032","140,111","11,532","125,837"
90,동구,"141,266","81,696","44,961","17,794","10,811","73,566","8,130","59,570"
121,북구,"139,347","79,777","42,353","24,380","6,698","73,431","6,346","59,570"
149,울주군,"165,014","92,066","57,676","19,762","5,654","83,092","8,974","72,948"


In [85]:
ulsan_6th = ulsan_6th.assign(
    시도='울산광역시'
)[['시도'] + ulsan_6th.columns.tolist()]

In [86]:
ulsan_6th.loc[0, '구시군'] = '합계'

In [87]:
ulsan_6th.head(3)

,시도,구시군,선거인수,투표수,득표수_1_새누리당_김기현,득표수_4_정의당_조승수,득표수_5_노동당_이갑용,득표수_계,무효투표수,기권수
0,울산광역시,합계,"912,325","511,881","306,311","123,736","38,107","468,154","43,727","400,444"
1,울산광역시,중구,"189,218","106,699","66,129","24,913","6,912","97,954","8,745","82,519"
44,울산광역시,남구,"277,480","151,643","95,192","36,887","8,032","140,111","11,532","125,837"


In [88]:
ulsan_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 149
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   시도              6 non-null      object
 1   구시군             6 non-null      object
 2   선거인수            6 non-null      object
 3   투표수             6 non-null      object
 4   득표수_1_새누리당_김기현  6 non-null      object
 5   득표수_4_정의당_조승수   6 non-null      object
 6   득표수_5_노동당_이갑용   6 non-null      object
 7   득표수_계           6 non-null      object
 8   무효투표수           6 non-null      object
 9   기권수             6 non-null      object
dtypes: object(10)
memory usage: 700.0+ bytes


In [89]:
ulsan_6th = ulsan_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [90]:
ulsan_6th_with_total = ulsan_6th

In [91]:
ulsan_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_김기현,득표수_4_정의당_조승수,득표수_5_노동당_이갑용,득표수_계,무효투표수,기권수
0,울산광역시,합계,912325,511881,306311,123736,38107,468154,43727,400444
1,울산광역시,중구,189218,106699,66129,24913,6912,97954,8745,82519
44,울산광역시,남구,277480,151643,95192,36887,8032,140111,11532,125837
90,울산광역시,동구,141266,81696,44961,17794,10811,73566,8130,59570
121,울산광역시,북구,139347,79777,42353,24380,6698,73431,6346,59570
149,울산광역시,울주군,165014,92066,57676,19762,5654,83092,8974,72948


In [92]:
ulsan_6th_with_total.to_csv("temp1_governor_ulsan_6.csv", index=False, encoding="utf-8-sig")


## Sejong (5)


In [93]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_sejong = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%A0%9C6%ED%9A%8C%20%EC%A0%84%EA%B5%AD%EB%8F%99%EC%8B%9C%EC%A7%80%EB%B0%A9%EC%84%A0%EA%B1%B0%20%EC%9D%8D%EB%A9%B4%EB%8F%99%EB%B3%84%20%EA%B0%9C%ED%91%9C%EC%9E%90%EB%A3%8C(%EC%84%B8%EC%A2%85).xlsx'

sejong_6th = process_6th_governor_election5(
    file_path_or_url = blob_url6_sejong,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합  계',
    add_sheet_name_column = False,
    sheet_names = '세종시장선거'  # 시장 선거 데이터만
)

파일의 전체 시트: 4개
시트 목록: ['세종시장선거', '세종시교육감선거', '세종시의원선거', '비례대표세종시의원선거']

선택된 시트 1개를 처리합니다: ['세종시장선거']

시트 1/1: '세종시장선거' 처리 중...
데이터 행 수: 40
필터링 적용: '읍면동명' == '합  계' → 1행
'세종시장선거' 시트 처리 완료: 1행 추가됨

최종 결과: 총 1행의 데이터


In [94]:
sejong_6th

,읍면동명,구분,선거인수,투표수,유 효 투 표 수 _새누리당\n유한식,새정치민주연합\n이춘희,계,무 효\n투표수,기권수
0,합 계,NaN,101559,63629,26451,36203,62654,975,37930


In [95]:
sejong_6th.columns.tolist()

['읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '유 효 투 표 수 _새누리당\n유한식',
 '새정치민주연합\n이춘희',
 '계',
 '무   효\n투표수',
 '기권수']

In [96]:
rename_sejong = {
    '읍면동명': '구시군',
    '유 효 투 표 수 _새누리당\n유한식': '득표수_1_새누리당_유한식',
    '새정치민주연합\n이춘희': '득표수_2_새정치민주연합_이춘희',
    '계': '득표수_계',
    '무   효\n투표수': '무효투표수'
    }

In [97]:
sejong_6th = sejong_6th.rename(columns=rename_sejong).drop(columns=['구분'])
sejong_6th

,구시군,선거인수,투표수,득표수_1_새누리당_유한식,득표수_2_새정치민주연합_이춘희,득표수_계,무효투표수,기권수
0,합 계,101559,63629,26451,36203,62654,975,37930


In [98]:
sejong_6th = sejong_6th.assign(
    시도='세종특별자치시'
)[['시도'] + sejong_6th.columns.tolist()]

In [99]:
sejong_6th.loc[0, '구시군'] = '합계'

In [100]:
sejong_6th

,시도,구시군,선거인수,투표수,득표수_1_새누리당_유한식,득표수_2_새정치민주연합_이춘희,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,101559,63629,26451,36203,62654,975,37930


In [101]:
sejong_6th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 1 non-null      object
 1   구시군                1 non-null      object
 2   선거인수               1 non-null      object
 3   투표수                1 non-null      object
 4   득표수_1_새누리당_유한식     1 non-null      object
 5   득표수_2_새정치민주연합_이춘희  1 non-null      object
 6   득표수_계              1 non-null      object
 7   무효투표수              1 non-null      object
 8   기권수                1 non-null      object
dtypes: object(9)
memory usage: 204.0+ bytes


In [102]:
sejong_6th = sejong_6th.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
)

In [103]:
sejong_6th_with_total = sejong_6th

In [104]:
sejong_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_유한식,득표수_2_새정치민주연합_이춘희,득표수_계,무효투표수,기권수
0,세종특별자치시,합계,101559,63629,26451,36203,62654,975,37930


In [105]:
sejong_6th_with_total.to_csv("temp1_governor_sejong_6.csv", index=False, encoding="utf-8-sig")


## Gyeonggi (1)


In [106]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_gyeonggi = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%9D%8D%EB%A9%B4%EB%8F%99%EB%B3%84%EA%B0%9C%ED%91%9C%EA%B2%B0%EA%B3%BC-%EA%B2%BD%EA%B8%B0%EB%8F%84%EC%A7%80%EC%82%AC.xlsx'

gyeonggi_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_gyeonggi,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합계'
)

헤더 행: 3행 ~ 4행
생성된 컬럼 수: 10
데이터 행 수: 1826

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
남경필
6: 새정치민주연합
김진표
7: 계
8: 무효
투표수
9: 기권수

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 1826행 → 필터링 후: 44행
'읍면동명' == '합계'인 행만 추출


In [107]:
gyeonggi_6th

,위원회명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n남경필,새정치민주연합\n김진표,계,무효\n투표수,기권수
0,수원시장안구,합계,NaN,"236,664","135,841","64,235","68,152","132,387","3,454","100,823"
34,수원시권선구,합계,NaN,"230,904","121,183","58,441","59,901","118,342","2,841","109,721"
68,수원시팔달구,합계,NaN,"203,541","105,744","54,133","48,962","103,095","2,649","97,797"
105,수원시영통구,합계,NaN,"240,776","142,435","58,276","81,424","139,700","2,735","98,341"
136,성남시수정구,합계,NaN,"191,794","97,639","43,871","50,514","94,385","3,254","94,155"
188,성남시중원구,합계,NaN,"211,683","109,445","49,739","55,787","105,526","3,919","102,238"
225,성남시분당구,합계,NaN,"389,934","241,915","123,412","114,263","237,675","4,240","148,019"
292,의정부시,합계,NaN,"345,708","171,491","86,274","80,456","166,730","4,761","174,217"
341,안양시만안구,합계,NaN,"205,999","113,522","56,134","54,398","110,532","2,990","92,477"
387,안양시동안구,합계,NaN,"280,346","168,487","78,311","86,376","164,687","3,800","111,859"


In [108]:
gyeonggi_6th.columns.tolist()

['위원회명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n남경필',
 '새정치민주연합\n김진표',
 '계',
 '무효\n투표수',
 '기권수']

In [109]:
rename_gyeonggi = {
    '위원회명': '구시군',
    '후보자별 득표수_새누리당\n남경필': '득표수_1_새누리당_남경필',
    '새정치민주연합\n김진표': '득표수_2_새정치민주연합_김진표',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [110]:
gyeonggi_6th = gyeonggi_6th.rename(columns=rename_gyeonggi).drop(columns=['읍면동명', '구분'])
gyeonggi_6th

,구시군,선거인수,투표수,득표수_1_새누리당_남경필,득표수_2_새정치민주연합_김진표,득표수_계,무효투표수,기권수
0,수원시장안구,"236,664","135,841","64,235","68,152","132,387","3,454","100,823"
34,수원시권선구,"230,904","121,183","58,441","59,901","118,342","2,841","109,721"
68,수원시팔달구,"203,541","105,744","54,133","48,962","103,095","2,649","97,797"
105,수원시영통구,"240,776","142,435","58,276","81,424","139,700","2,735","98,341"
136,성남시수정구,"191,794","97,639","43,871","50,514","94,385","3,254","94,155"
188,성남시중원구,"211,683","109,445","49,739","55,787","105,526","3,919","102,238"
225,성남시분당구,"389,934","241,915","123,412","114,263","237,675","4,240","148,019"
292,의정부시,"345,708","171,491","86,274","80,456","166,730","4,761","174,217"
341,안양시만안구,"205,999","113,522","56,134","54,398","110,532","2,990","92,477"
387,안양시동안구,"280,346","168,487","78,311","86,376","164,687","3,800","111,859"


In [111]:
gyeonggi_6th = gyeonggi_6th.assign(
    시도='경기도'
)[['시도'] + gyeonggi_6th.columns.tolist()]

In [112]:
gyeonggi_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_남경필,득표수_2_새정치민주연합_김진표,득표수_계,무효투표수,기권수
0,경기도,수원시장안구,"236,664","135,841","64,235","68,152","132,387","3,454","100,823"
34,경기도,수원시권선구,"230,904","121,183","58,441","59,901","118,342","2,841","109,721"
68,경기도,수원시팔달구,"203,541","105,744","54,133","48,962","103,095","2,649","97,797"
105,경기도,수원시영통구,"240,776","142,435","58,276","81,424","139,700","2,735","98,341"
136,경기도,성남시수정구,"191,794","97,639","43,871","50,514","94,385","3,254","94,155"


In [113]:
gyeonggi_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 0 to 1804
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 44 non-null     object
 1   구시군                44 non-null     object
 2   선거인수               44 non-null     object
 3   투표수                44 non-null     object
 4   득표수_1_새누리당_남경필     44 non-null     object
 5   득표수_2_새정치민주연합_김진표  44 non-null     object
 6   득표수_계              44 non-null     object
 7   무효투표수              44 non-null     object
 8   기권수                44 non-null     object
dtypes: object(9)
memory usage: 4.5+ KB


In [114]:
gyeonggi_6th = gyeonggi_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [115]:
# 수치형 열만 합계 구하기
summary_row = gyeonggi_6th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '경기도')

# summary_row를 맨 위에 붙이기
gyeonggi_6th_with_total = pd.concat([summary_row, gyeonggi_6th], ignore_index=True)

In [116]:
gyeonggi_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_남경필,득표수_2_새정치민주연합_김진표,득표수_계,무효투표수,기권수
0,경기도,합계,9679317,5156691,2524981,2481824,5006805,149886,4522626
1,경기도,수원시장안구,236664,135841,64235,68152,132387,3454,100823
2,경기도,수원시권선구,230904,121183,58441,59901,118342,2841,109721
3,경기도,수원시팔달구,203541,105744,54133,48962,103095,2649,97797
4,경기도,수원시영통구,240776,142435,58276,81424,139700,2735,98341
5,경기도,성남시수정구,191794,97639,43871,50514,94385,3254,94155
6,경기도,성남시중원구,211683,109445,49739,55787,105526,3919,102238
7,경기도,성남시분당구,389934,241915,123412,114263,237675,4240,148019
8,경기도,의정부시,345708,171491,86274,80456,166730,4761,174217
9,경기도,안양시만안구,205999,113522,56134,54398,110532,2990,92477


In [117]:
gyeonggi_6th_with_total.to_csv("temp1_governor_gyeonggi_6.csv", index=False, encoding="utf-8-sig")


## Gangwon (1)


In [118]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_gangwon = "https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EA%B0%95%EC%9B%90%EB%8F%84%EC%A7%80%EC%82%AC%EC%84%A0%EA%B1%B0.xlsx"

gangwon_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_gangwon,
    header_rows = (2, 4),
    filter_column = '읍면동명',
    filter_value = '합       계'
)

헤더 행: 2행 ~ 4행
생성된 컬럼 수: 12
데이터 행 수: 637

생성된 컬럼명 (처음 10개):
0: 구시군명
1: 읍면동명
2: 구분
3: 선거인수
(가+나+다)
4: 투표수
(가+나)
5: 유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당
최흥집
6: 새정치
민주연합
최문순
7: 통합진보당
이승재
8: 계
9: 무효
투표수
(나)

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 637행 → 필터링 후: 18행
'읍면동명' == '합       계'인 행만 추출


In [119]:
gangwon_6th

,구시군명,읍면동명,구분,선거인수\n(가+나+다),투표수\n(가+나),유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당\n최흥집,새정치\n민주연합\n최문순,통합진보당\n이승재,계,무효\n투표수\n(나),기권수\n(다),비고
0,강원도,합 계,NaN,1255469,781359,369201,381338,15774,766313,15046,474110,NaN
1,춘천시,합 계,NaN,220197,131007,50016,77737,1857,129610,1397,89190,NaN
80,원주시,합 계,NaN,257088,142997,64877,73995,2442,141314,1683,114091,NaN
159,강릉시,합 계,NaN,176430,104326,60251,40526,1736,102513,1813,72104,NaN
226,동해시,합 계,NaN,75796,45629,23041,20786,942,44769,860,30167,NaN
260,삼척시,합 계,NaN,61597,42406,20875,18782,1514,41171,1235,19191,NaN
300,태백시,합 계,NaN,40070,27089,13097,12746,449,26292,797,12981,NaN
359,속초시,합 계,NaN,66777,39108,18718,18821,694,38233,875,27669,NaN
387,고성군,합 계,NaN,26235,19179,9569,8480,596,18645,534,7056,NaN
406,양양군,합 계,NaN,23787,17549,8761,7967,383,17111,438,6238,NaN


In [120]:
gangwon_6th.columns.tolist()

['구시군명',
 '읍면동명',
 '구분',
 '선거인수\n(가+나+다)',
 '투표수\n(가+나)',
 '유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당\n최흥집',
 '새정치\n민주연합\n최문순',
 '통합진보당\n이승재',
 '계',
 '무효\n투표수\n(나)',
 '기권수\n(다)',
 '비고']

In [121]:
rename_gangwon = {
    '구시군명': '구시군',
    '선거인수\n(가+나+다)': '선거인수',
    '투표수\n(가+나)': '투표수',
    '유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당\n최흥집': '득표수_1_새누리당_최흥집',
    '새정치\n민주연합\n최문순': '득표수_2_새정치민주연합_최문순',
    '통합진보당\n이승재': '득표수_3_통합진보당_이승재',
    '계': '득표수_계',
    '무효\n투표수\n(나)': '무효투표수',
    '기권수\n(다)': '기권수'
    }

In [122]:
gangwon_6th = gangwon_6th.rename(columns=rename_gangwon).drop(columns=['읍면동명', '구분', '비고'])
gangwon_6th

,구시군,선거인수,투표수,득표수_1_새누리당_최흥집,득표수_2_새정치민주연합_최문순,득표수_3_통합진보당_이승재,득표수_계,무효투표수,기권수
0,강원도,1255469,781359,369201,381338,15774,766313,15046,474110
1,춘천시,220197,131007,50016,77737,1857,129610,1397,89190
80,원주시,257088,142997,64877,73995,2442,141314,1683,114091
159,강릉시,176430,104326,60251,40526,1736,102513,1813,72104
226,동해시,75796,45629,23041,20786,942,44769,860,30167
260,삼척시,61597,42406,20875,18782,1514,41171,1235,19191
300,태백시,40070,27089,13097,12746,449,26292,797,12981
359,속초시,66777,39108,18718,18821,694,38233,875,27669
387,고성군,26235,19179,9569,8480,596,18645,534,7056
406,양양군,23787,17549,8761,7967,383,17111,438,6238


In [123]:
gangwon_6th = gangwon_6th.assign(
    시도='강원도'
)[['시도'] + gangwon_6th.columns.tolist()]

In [124]:
gangwon_6th.loc[0, '구시군'] = '합계'

In [125]:
gangwon_6th

,시도,구시군,선거인수,투표수,득표수_1_새누리당_최흥집,득표수_2_새정치민주연합_최문순,득표수_3_통합진보당_이승재,득표수_계,무효투표수,기권수
0,강원도,합계,1255469,781359,369201,381338,15774,766313,15046,474110
1,강원도,춘천시,220197,131007,50016,77737,1857,129610,1397,89190
80,강원도,원주시,257088,142997,64877,73995,2442,141314,1683,114091
159,강원도,강릉시,176430,104326,60251,40526,1736,102513,1813,72104
226,강원도,동해시,75796,45629,23041,20786,942,44769,860,30167
260,강원도,삼척시,61597,42406,20875,18782,1514,41171,1235,19191
300,강원도,태백시,40070,27089,13097,12746,449,26292,797,12981
359,강원도,속초시,66777,39108,18718,18821,694,38233,875,27669
387,강원도,고성군,26235,19179,9569,8480,596,18645,534,7056
406,강원도,양양군,23787,17549,8761,7967,383,17111,438,6238


In [126]:
gangwon_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 0 to 612
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 18 non-null     object
 1   구시군                18 non-null     object
 2   선거인수               18 non-null     object
 3   투표수                18 non-null     object
 4   득표수_1_새누리당_최흥집     18 non-null     object
 5   득표수_2_새정치민주연합_최문순  18 non-null     object
 6   득표수_3_통합진보당_이승재    18 non-null     object
 7   득표수_계              18 non-null     object
 8   무효투표수              18 non-null     object
 9   기권수                18 non-null     object
dtypes: object(10)
memory usage: 2.1+ KB


In [127]:
gangwon_6th = gangwon_6th.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
)

In [128]:
gangwon_6th_with_total = gangwon_6th

In [129]:
gangwon_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_최흥집,득표수_2_새정치민주연합_최문순,득표수_3_통합진보당_이승재,득표수_계,무효투표수,기권수
0,강원도,합계,1255469,781359,369201,381338,15774,766313,15046,474110
1,강원도,춘천시,220197,131007,50016,77737,1857,129610,1397,89190
80,강원도,원주시,257088,142997,64877,73995,2442,141314,1683,114091
159,강원도,강릉시,176430,104326,60251,40526,1736,102513,1813,72104
226,강원도,동해시,75796,45629,23041,20786,942,44769,860,30167
260,강원도,삼척시,61597,42406,20875,18782,1514,41171,1235,19191
300,강원도,태백시,40070,27089,13097,12746,449,26292,797,12981
359,강원도,속초시,66777,39108,18718,18821,694,38233,875,27669
387,강원도,고성군,26235,19179,9569,8480,596,18645,534,7056
406,강원도,양양군,23787,17549,8761,7967,383,17111,438,6238


In [130]:
gangwon_6th_with_total.to_csv("temp1_governor_gangwon_6.csv", index=False, encoding="utf-8-sig")


## Chungbuk (5)


In [131]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_chungbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EB%8F%84%EC%A7%80%EC%82%AC.xls'

chungbuk_6th = process_6th_governor_election5(
    file_path_or_url = blob_url6_chungbuk,
    header_rows = (3, 4),
    filter_column = None,
    filter_value = None,
    add_sheet_name_column = False,
    sheet_names = '개표진행상황_위원회별합계'
)

파일의 전체 시트: 14개
시트 목록: ['개표진행상황_위원회별합계', '상당구', '흥덕구', '충주시', '제천시', '단양군', '청원군', '영동군', '보은군', '옥천군', '음성군', '진천군', '괴산군', '증평군']

선택된 시트 1개를 처리합니다: ['개표진행상황_위원회별합계']

시트 1/1: '개표진행상황_위원회별합계' 처리 중...
데이터 행 수: 15
'개표진행상황_위원회별합계' 시트 처리 완료: 15행 추가됨

최종 결과: 총 15행의 데이터


In [132]:
chungbuk_6th

,구시군명,선거인수,투표수,후보자별 득표수(득표율)_새누리당\n윤진식,새정치민주연합\n이시종,통합진보당\n신장호,계,무효\n투표수,기권수
0,합계,"1,261,119","741,049","346,152\n(47.68)","361,115\n(49.75)","18,590\n(2.56)","725,857","15,192","520,070"
1,청주시상당구,"194,201","107,458","51,082\n(48.10)","52,717\n(49.64)","2,382\n(2.24)","106,181","1,277","86,743"
2,청주시흥덕구,"328,448","178,070","79,126\n(44.91)","92,795\n(52.67)","4,259\n(2.41)","176,180","1,890","150,378"
3,충주시,"169,145","97,900","49,606\n(51.50)","45,133\n(46.86)","1,571\n(1.63)","96,310","1,590","71,245"
4,제천시,"112,013","67,732","31,934\n(48.26)","32,605\n(49.28)","1,619\n(2.44)","66,158","1,574","44,281"
5,단양군,"26,692","18,685","9,313\n(51.52)","8,268\n(45.74)",495\n(2.73),"18,076",609,"8,007"
6,청원군,"123,457","70,847","32,671\n(47.14)","34,629\n(49.96)","2,005\n(2.89)","69,305","1,542","52,610"
7,영동군,"43,012","30,700","15,150\n(51.23)","13,444\n(45.46)",974\n(3.29),"29,568","1,132","12,312"
8,보은군,"29,809","22,561","10,416\n(48.31)","10,369\n(48.09)",773\n(3.58),"21,558","1,003","7,248"
9,옥천군,"44,490","30,898","13,947\n(46.97)","14,869\n(50.08)",872\n(2.93),"29,688","1,210","13,592"


In [133]:
chungbuk_6th.columns.tolist()

['구시군명',
 '선거인수',
 '투표수',
 '후보자별 득표수(득표율)_새누리당\n윤진식',
 '새정치민주연합\n이시종',
 '통합진보당\n신장호',
 '계',
 '무효\n투표수',
 '기권수']

In [134]:
rename_chungbuk = {
    '구시군명': '구시군',
    '후보자별 득표수(득표율)_새누리당\n윤진식': '득표수_1_새누리당_윤진식',
    '새정치민주연합\n이시종': '득표수_2_새정치민주연합_이시종',
    '통합진보당\n신장호': '득표수_3_통합진보당_신장호',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [135]:
chungbuk_6th = chungbuk_6th.rename(columns=rename_chungbuk).iloc[:-1]
chungbuk_6th

,구시군,선거인수,투표수,득표수_1_새누리당_윤진식,득표수_2_새정치민주연합_이시종,득표수_3_통합진보당_신장호,득표수_계,무효투표수,기권수
0,합계,"1,261,119","741,049","346,152\n(47.68)","361,115\n(49.75)","18,590\n(2.56)","725,857","15,192","520,070"
1,청주시상당구,"194,201","107,458","51,082\n(48.10)","52,717\n(49.64)","2,382\n(2.24)","106,181","1,277","86,743"
2,청주시흥덕구,"328,448","178,070","79,126\n(44.91)","92,795\n(52.67)","4,259\n(2.41)","176,180","1,890","150,378"
3,충주시,"169,145","97,900","49,606\n(51.50)","45,133\n(46.86)","1,571\n(1.63)","96,310","1,590","71,245"
4,제천시,"112,013","67,732","31,934\n(48.26)","32,605\n(49.28)","1,619\n(2.44)","66,158","1,574","44,281"
5,단양군,"26,692","18,685","9,313\n(51.52)","8,268\n(45.74)",495\n(2.73),"18,076",609,"8,007"
6,청원군,"123,457","70,847","32,671\n(47.14)","34,629\n(49.96)","2,005\n(2.89)","69,305","1,542","52,610"
7,영동군,"43,012","30,700","15,150\n(51.23)","13,444\n(45.46)",974\n(3.29),"29,568","1,132","12,312"
8,보은군,"29,809","22,561","10,416\n(48.31)","10,369\n(48.09)",773\n(3.58),"21,558","1,003","7,248"
9,옥천군,"44,490","30,898","13,947\n(46.97)","14,869\n(50.08)",872\n(2.93),"29,688","1,210","13,592"


In [136]:
chungbuk_6th = chungbuk_6th.assign(
    시도='충청북도'
)[['시도'] + chungbuk_6th.columns.tolist()]

In [137]:
chungbuk_6th.head(3)

,시도,구시군,선거인수,투표수,득표수_1_새누리당_윤진식,득표수_2_새정치민주연합_이시종,득표수_3_통합진보당_신장호,득표수_계,무효투표수,기권수
0,충청북도,합계,"1,261,119","741,049","346,152\n(47.68)","361,115\n(49.75)","18,590\n(2.56)","725,857","15,192","520,070"
1,충청북도,청주시상당구,"194,201","107,458","51,082\n(48.10)","52,717\n(49.64)","2,382\n(2.24)","106,181","1,277","86,743"
2,충청북도,청주시흥덕구,"328,448","178,070","79,126\n(44.91)","92,795\n(52.67)","4,259\n(2.41)","176,180","1,890","150,378"


In [138]:
chungbuk_6th.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 14 non-null     object
 1   구시군                14 non-null     object
 2   선거인수               14 non-null     object
 3   투표수                14 non-null     object
 4   득표수_1_새누리당_윤진식     14 non-null     object
 5   득표수_2_새정치민주연합_이시종  14 non-null     object
 6   득표수_3_통합진보당_신장호    14 non-null     object
 7   득표수_계              14 non-null     object
 8   무효투표수              14 non-null     object
 9   기권수                14 non-null     object
dtypes: object(10)
memory usage: 1.2+ KB


In [139]:
chungbuk_6th = chungbuk_6th.apply(
    lambda col: col.str.replace(r'\n.*$', '', regex=True)
    if col.dtype == 'object' and col.astype(str).str.contains('\n').any()
    else col
).apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
)

In [140]:
chungbuk_6th_with_total = chungbuk_6th

In [141]:
chungbuk_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_윤진식,득표수_2_새정치민주연합_이시종,득표수_3_통합진보당_신장호,득표수_계,무효투표수,기권수
0,충청북도,합계,1261119,741049,346152,361115,18590,725857,15192,520070
1,충청북도,청주시상당구,194201,107458,51082,52717,2382,106181,1277,86743
2,충청북도,청주시흥덕구,328448,178070,79126,92795,4259,176180,1890,150378
3,충청북도,충주시,169145,97900,49606,45133,1571,96310,1590,71245
4,충청북도,제천시,112013,67732,31934,32605,1619,66158,1574,44281
5,충청북도,단양군,26692,18685,9313,8268,495,18076,609,8007
6,충청북도,청원군,123457,70847,32671,34629,2005,69305,1542,52610
7,충청북도,영동군,43012,30700,15150,13444,974,29568,1132,12312
8,충청북도,보은군,29809,22561,10416,10369,773,21558,1003,7248
9,충청북도,옥천군,44490,30898,13947,14869,872,29688,1210,13592


In [142]:
chungbuk_6th_with_total.to_csv("temp1_governor_chungbuk_6.csv", index=False, encoding="utf-8-sig")


## Chungnam (1)


In [143]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_chungnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%B6%A9%EC%B2%AD%EB%82%A8%EB%8F%84.xls'

chungnam_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_chungnam,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합계'
)

헤더 행: 3행 ~ 4행
생성된 컬럼 수: 11
데이터 행 수: 685

생성된 컬럼명 (처음 10개):
0: 위원회명
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
정진석
6: 새정치민주연합
안희정
7: 무소속
김기문
8: 계
9: 무효
투표수

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 685행 → 필터링 후: 16행
'읍면동명' == '합계'인 행만 추출


In [144]:
chungnam_6th

,위원회명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n정진석,새정치민주연합\n안희정,무소속\n김기문,계,무효\n투표수,기권수
0,천안시\n서북구,합계,NaN,223113,103339,40623,58379,3198,102200,1139,119774
40,천안시\n동남구,합계,NaN,236496,117049,49771,62796,2987,115554,1495,119447
98,공주시,합계,NaN,96439,57596,26855,27730,1519,56104,1492,38843
150,보령시,합계,NaN,86603,55365,25254,25451,2550,53255,2110,31238
202,아산시,합계,NaN,222224,113433,47139,60927,3356,111422,2011,108791
257,서산시,합계,NaN,130906,70754,30125,36161,2781,69067,1687,60152
306,태안군,합계,NaN,53331,35682,16046,15507,2419,33972,1710,17649
334,금산군,합계,NaN,46813,30036,12522,14725,1612,28859,1177,16777
368,논산시,합계,NaN,103962,62061,21888,36205,2037,60130,1931,41901
417,계룡시,합계,NaN,29874,19557,8353,10274,664,19291,266,10317


In [145]:
chungnam_6th.columns.tolist()

['위원회명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n정진석',
 '새정치민주연합\n안희정',
 '무소속\n김기문',
 '계',
 '무효\n투표수',
 '기권수']

In [146]:
rename_chungnam = {
    '위원회명': '구시군',
    '후보자별 득표수_새누리당\n정진석': '득표수_1_새누리당_정진석',
    '새정치민주연합\n안희정': '득표수_2_새정치민주연합_안희정',
    '무소속\n김기문': '득표수_4_무소속_김기문',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [147]:
chungnam_6th = chungnam_6th.rename(columns=rename_chungnam).drop(columns=['읍면동명', '구분'])
chungnam_6th

,구시군,선거인수,투표수,득표수_1_새누리당_정진석,득표수_2_새정치민주연합_안희정,득표수_4_무소속_김기문,득표수_계,무효투표수,기권수
0,천안시\n서북구,223113,103339,40623,58379,3198,102200,1139,119774
40,천안시\n동남구,236496,117049,49771,62796,2987,115554,1495,119447
98,공주시,96439,57596,26855,27730,1519,56104,1492,38843
150,보령시,86603,55365,25254,25451,2550,53255,2110,31238
202,아산시,222224,113433,47139,60927,3356,111422,2011,108791
257,서산시,130906,70754,30125,36161,2781,69067,1687,60152
306,태안군,53331,35682,16046,15507,2419,33972,1710,17649
334,금산군,46813,30036,12522,14725,1612,28859,1177,16777
368,논산시,103962,62061,21888,36205,2037,60130,1931,41901
417,계룡시,29874,19557,8353,10274,664,19291,266,10317


In [148]:
chungnam_6th = chungnam_6th.assign(
    시도='충청남도'
)[['시도'] + chungnam_6th.columns.tolist()]

In [149]:
chungnam_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_정진석,득표수_2_새정치민주연합_안희정,득표수_4_무소속_김기문,득표수_계,무효투표수,기권수
0,충청남도,천안시\n서북구,223113,103339,40623,58379,3198,102200,1139,119774
40,충청남도,천안시\n동남구,236496,117049,49771,62796,2987,115554,1495,119447
98,충청남도,공주시,96439,57596,26855,27730,1519,56104,1492,38843
150,충청남도,보령시,86603,55365,25254,25451,2550,53255,2110,31238
202,충청남도,아산시,222224,113433,47139,60927,3356,111422,2011,108791


In [150]:
chungnam_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 645
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 16 non-null     object
 1   구시군                16 non-null     object
 2   선거인수               16 non-null     object
 3   투표수                16 non-null     object
 4   득표수_1_새누리당_정진석     16 non-null     object
 5   득표수_2_새정치민주연합_안희정  16 non-null     object
 6   득표수_4_무소속_김기문      16 non-null     object
 7   득표수_계              16 non-null     object
 8   무효투표수              16 non-null     object
 9   기권수                16 non-null     object
dtypes: object(10)
memory usage: 1.9+ KB


In [151]:
chungnam_6th = chungnam_6th.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
).assign(구시군=lambda df: df['구시군'].str.replace('\n', '', regex=False))

In [152]:
# 수치형 열만 합계 구하기
summary_row = chungnam_6th.select_dtypes(include='number').sum().to_frame().T

# 시도와 구시군 값 추가
summary_row.insert(0, '구시군', '합계')
summary_row.insert(0, '시도', '충청남도')

# summary_row를 맨 위에 붙이기
chungnam_6th_with_total = pd.concat([summary_row, chungnam_6th], ignore_index=True)

In [153]:
chungnam_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_정진석,득표수_2_새정치민주연합_안희정,득표수_4_무소속_김기문,득표수_계,무효투표수,기권수
0,충청남도,합계,1644554,916206,392315,465994,34204,892513,23693,728348
1,충청남도,천안시서북구,223113,103339,40623,58379,3198,102200,1139,119774
2,충청남도,천안시동남구,236496,117049,49771,62796,2987,115554,1495,119447
3,충청남도,공주시,96439,57596,26855,27730,1519,56104,1492,38843
4,충청남도,보령시,86603,55365,25254,25451,2550,53255,2110,31238
5,충청남도,아산시,222224,113433,47139,60927,3356,111422,2011,108791
6,충청남도,서산시,130906,70754,30125,36161,2781,69067,1687,60152
7,충청남도,태안군,53331,35682,16046,15507,2419,33972,1710,17649
8,충청남도,금산군,46813,30036,12522,14725,1612,28859,1177,16777
9,충청남도,논산시,103962,62061,21888,36205,2037,60130,1931,41901


In [154]:
chungnam_6th_with_total.to_csv("temp1_governor_chungnam_6.csv", index=False, encoding="utf-8-sig")


## Jeonbuk (1)


In [155]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_jeonbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%A0%84%EB%9D%BC%EB%B6%81%EB%8F%84.xls'

jeonbuk_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_jeonbuk,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = '합계'
)

헤더 행: 3행 ~ 4행
생성된 컬럼 수: 11
데이터 행 수: 812

생성된 컬럼명 (처음 10개):
0: 구시군명
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
박철곤
6: 새정치민주연합
송하진
7: 통합진보당
이광석
8: 계
9: 무효
투표수

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 812행 → 필터링 후: 16행
'읍면동명' == '합계'인 행만 추출


In [156]:
jeonbuk_6th

,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n박철곤,새정치민주연합\n송하진,통합진보당\n이광석,계,무효\n투표수,기권수
0,전라북도,합계,NaN,1503242,900029,177172,599654,89337,866163,33866,603213
1,완산구,합계,NaN,"283,872","159,841","39,423","99,971","17,298","156,692","3,149","124,031"
61,덕진구,합계,NaN,"221,516","123,206","29,005","77,702","14,026","120,733","2,473","98,310"
112,군산시,합계,NaN,"220,810","117,779","19,923","83,123","10,831","113,877","3,902","103,031"
199,익산시,합계,NaN,"244,122","129,480","22,697","91,000","11,226","124,923","4,557","114,642"
292,정읍시,합계,NaN,"96,960","61,781","9,104","44,302","5,359","58,765","3,016","35,179"
367,남원시,합계,NaN,"70,411","47,558","8,103","32,868","4,594","45,565","1,993","22,853"
442,김제시,합계,NaN,"77,336","51,495","7,317","37,675","3,766","48,758","2,737","25,841"
505,완주군,합계,NaN,"70,666","44,750","9,986","26,815","5,816","42,617","2,133","25,916"
550,진안군,합계,NaN,"23,289","18,439","6,355","9,290","1,789","17,434","1,005","4,850"


In [157]:
jeonbuk_6th.columns.tolist()

['구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n박철곤',
 '새정치민주연합\n송하진',
 '통합진보당\n이광석',
 '계',
 '무효\n투표수',
 '기권수']

In [158]:
rename_jeonbuk = {
    '구시군명': '구시군',
    '후보자별 득표수_새누리당\n박철곤': '득표수_1_새누리당_박철곤',
    '새정치민주연합\n송하진': '득표수_2_새정치민주연합_송하진',
    '통합진보당\n이광석': '득표수_3_통합진보당_이광석',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [159]:
jeonbuk_6th = jeonbuk_6th.rename(columns=rename_jeonbuk).drop(columns=['읍면동명', '구분'])
jeonbuk_6th

,구시군,선거인수,투표수,득표수_1_새누리당_박철곤,득표수_2_새정치민주연합_송하진,득표수_3_통합진보당_이광석,득표수_계,무효투표수,기권수
0,전라북도,1503242,900029,177172,599654,89337,866163,33866,603213
1,완산구,"283,872","159,841","39,423","99,971","17,298","156,692","3,149","124,031"
61,덕진구,"221,516","123,206","29,005","77,702","14,026","120,733","2,473","98,310"
112,군산시,"220,810","117,779","19,923","83,123","10,831","113,877","3,902","103,031"
199,익산시,"244,122","129,480","22,697","91,000","11,226","124,923","4,557","114,642"
292,정읍시,"96,960","61,781","9,104","44,302","5,359","58,765","3,016","35,179"
367,남원시,"70,411","47,558","8,103","32,868","4,594","45,565","1,993","22,853"
442,김제시,"77,336","51,495","7,317","37,675","3,766","48,758","2,737","25,841"
505,완주군,"70,666","44,750","9,986","26,815","5,816","42,617","2,133","25,916"
550,진안군,"23,289","18,439","6,355","9,290","1,789","17,434","1,005","4,850"


In [160]:
jeonbuk_6th = jeonbuk_6th.assign(
    시도='전라북도'
)[['시도'] + jeonbuk_6th.columns.tolist()]

In [161]:
jeonbuk_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박철곤,득표수_2_새정치민주연합_송하진,득표수_3_통합진보당_이광석,득표수_계,무효투표수,기권수
0,전라북도,전라북도,1503242,900029,177172,599654,89337,866163,33866,603213
1,전라북도,완산구,"283,872","159,841","39,423","99,971","17,298","156,692","3,149","124,031"
61,전라북도,덕진구,"221,516","123,206","29,005","77,702","14,026","120,733","2,473","98,310"
112,전라북도,군산시,"220,810","117,779","19,923","83,123","10,831","113,877","3,902","103,031"
199,전라북도,익산시,"244,122","129,480","22,697","91,000","11,226","124,923","4,557","114,642"


In [162]:
jeonbuk_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 769
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 16 non-null     object
 1   구시군                16 non-null     object
 2   선거인수               16 non-null     object
 3   투표수                16 non-null     object
 4   득표수_1_새누리당_박철곤     16 non-null     object
 5   득표수_2_새정치민주연합_송하진  16 non-null     object
 6   득표수_3_통합진보당_이광석    16 non-null     object
 7   득표수_계              16 non-null     object
 8   무효투표수              16 non-null     object
 9   기권수                16 non-null     object
dtypes: object(10)
memory usage: 1.9+ KB


In [163]:
jeonbuk_6th = jeonbuk_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
).assign(
    구시군=lambda df: df['구시군']
        .replace({'전라북도': '합계', '완산구': '전주시완산구', '덕진구': '전주시덕진구'})
)

In [164]:
jeonbuk_6th.head(5)

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박철곤,득표수_2_새정치민주연합_송하진,득표수_3_통합진보당_이광석,득표수_계,무효투표수,기권수
0,전라북도,합계,1503242,900029,177172,599654,89337,866163,33866,603213
1,전라북도,전주시완산구,283872,159841,39423,99971,17298,156692,3149,124031
61,전라북도,전주시덕진구,221516,123206,29005,77702,14026,120733,2473,98310
112,전라북도,군산시,220810,117779,19923,83123,10831,113877,3902,103031
199,전라북도,익산시,244122,129480,22697,91000,11226,124923,4557,114642


In [165]:
jeonbuk_6th_with_total = jeonbuk_6th

In [166]:
jeonbuk_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_박철곤,득표수_2_새정치민주연합_송하진,득표수_3_통합진보당_이광석,득표수_계,무효투표수,기권수
0,전라북도,합계,1503242,900029,177172,599654,89337,866163,33866,603213
1,전라북도,전주시완산구,283872,159841,39423,99971,17298,156692,3149,124031
61,전라북도,전주시덕진구,221516,123206,29005,77702,14026,120733,2473,98310
112,전라북도,군산시,220810,117779,19923,83123,10831,113877,3902,103031
199,전라북도,익산시,244122,129480,22697,91000,11226,124923,4557,114642
292,전라북도,정읍시,96960,61781,9104,44302,5359,58765,3016,35179
367,전라북도,남원시,70411,47558,8103,32868,4594,45565,1993,22853
442,전라북도,김제시,77336,51495,7317,37675,3766,48758,2737,25841
505,전라북도,완주군,70666,44750,9986,26815,5816,42617,2133,25916
550,전라북도,진안군,23289,18439,6355,9290,1789,17434,1005,4850


In [167]:
jeonbuk_6th_with_total.to_csv("temp1_governor_jeonbuk_6.csv", index=False, encoding="utf-8-sig")


## Jeonnam (1)


In [168]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_jeonnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%A0%84%EB%9D%BC%EB%82%A8%EB%8F%84.xls'

jeonnam_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_jeonnam,
    header_rows = (3, 5),
    filter_column = '읍면동명',
    filter_value = '합계'
)

헤더 행: 3행 ~ 5행
생성된 컬럼 수: 11
데이터 행 수: 980

생성된 컬럼명 (처음 10개):
0: 구시군명
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
이중효
6: 새정치민주연합
이낙연
7: 통합진보당
이성수
8: 계
9: 무효
투표수

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 980행 → 필터링 후: 23행
'읍면동명' == '합계'인 행만 추출


In [169]:
jeonnam_6th

,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n이중효,새정치민주연합\n이낙연,통합진보당\n이성수,계,무효\n투표수,기권수
0,전라남도,합계,NaN,1549440,1015688,92549,755233,120868,968650,47038,533752
1,목포시,합계,NaN,"185,570","103,181","8,017","80,860","10,840","99,717","3,464","82,389"
74,여수시,합계,NaN,"232,635","135,390","11,394","104,927","15,608","131,929","3,461","97,245"
159,순천시,합계,NaN,"214,889","126,320","10,695","95,005","16,777","122,477","3,843","88,569"
235,나주시,합계,NaN,"76,122","51,515","4,157","38,556","6,067","48,780","2,735","24,607"
299,광양시,합계,NaN,"115,763","73,537","8,669","52,699","9,775","71,143","2,394","42,226"
339,담양군,합계,NaN,"41,046","29,086","2,627","22,183","3,004","27,814","1,272","11,960"
379,장성군,합계,NaN,"39,166","28,305","3,097","20,673","3,135","26,905","1,400","10,861"
416,곡성군,합계,NaN,"26,872","20,627","1,976","14,497","2,835","19,308","1,319","6,245"
453,구례군,합계,NaN,"23,363","19,041","2,351","12,759","2,796","17,906","1,135","4,322"


In [170]:
jeonnam_6th.columns.tolist()

['구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n이중효',
 '새정치민주연합\n이낙연',
 '통합진보당\n이성수',
 '계',
 '무효\n투표수',
 '기권수']

In [171]:
rename_jeonnam = {
    '구시군명': '구시군',
    '후보자별 득표수_새누리당\n이중효': '득표수_1_새누리당_이중효',
    '새정치민주연합\n이낙연': '득표수_2_새정치민주연합_이낙연',
    '통합진보당\n이성수': '득표수_3_통합진보당_이성수',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [172]:
jeonnam_6th = jeonnam_6th.rename(columns=rename_jeonnam).drop(columns=['읍면동명', '구분'])
jeonnam_6th

,구시군,선거인수,투표수,득표수_1_새누리당_이중효,득표수_2_새정치민주연합_이낙연,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수
0,전라남도,1549440,1015688,92549,755233,120868,968650,47038,533752
1,목포시,"185,570","103,181","8,017","80,860","10,840","99,717","3,464","82,389"
74,여수시,"232,635","135,390","11,394","104,927","15,608","131,929","3,461","97,245"
159,순천시,"214,889","126,320","10,695","95,005","16,777","122,477","3,843","88,569"
235,나주시,"76,122","51,515","4,157","38,556","6,067","48,780","2,735","24,607"
299,광양시,"115,763","73,537","8,669","52,699","9,775","71,143","2,394","42,226"
339,담양군,"41,046","29,086","2,627","22,183","3,004","27,814","1,272","11,960"
379,장성군,"39,166","28,305","3,097","20,673","3,135","26,905","1,400","10,861"
416,곡성군,"26,872","20,627","1,976","14,497","2,835","19,308","1,319","6,245"
453,구례군,"23,363","19,041","2,351","12,759","2,796","17,906","1,135","4,322"


In [173]:
jeonnam_6th = jeonnam_6th.assign(
    시도='전라남도'
)[['시도'] + jeonnam_6th.columns.tolist()]

In [174]:
jeonnam_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_이중효,득표수_2_새정치민주연합_이낙연,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수
0,전라남도,전라남도,1549440,1015688,92549,755233,120868,968650,47038,533752
1,전라남도,목포시,"185,570","103,181","8,017","80,860","10,840","99,717","3,464","82,389"
74,전라남도,여수시,"232,635","135,390","11,394","104,927","15,608","131,929","3,461","97,245"
159,전라남도,순천시,"214,889","126,320","10,695","95,005","16,777","122,477","3,843","88,569"
235,전라남도,나주시,"76,122","51,515","4,157","38,556","6,067","48,780","2,735","24,607"


In [175]:
jeonnam_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 0 to 934
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 23 non-null     object
 1   구시군                23 non-null     object
 2   선거인수               23 non-null     object
 3   투표수                23 non-null     object
 4   득표수_1_새누리당_이중효     23 non-null     object
 5   득표수_2_새정치민주연합_이낙연  23 non-null     object
 6   득표수_3_통합진보당_이성수    23 non-null     object
 7   득표수_계              23 non-null     object
 8   무효투표수              23 non-null     object
 9   기권수                23 non-null     object
dtypes: object(10)
memory usage: 2.5+ KB


In [176]:
jeonnam_6th = jeonnam_6th.apply(
    lambda col: col.astype(str).str.replace(',', '').astype(int)
    if col.dtypes == 'object' and col.astype(str).str.fullmatch(r'[\d,]+').all()
    else col
).assign(
    구시군=lambda df: df['구시군']
        .replace({'전라남도': '합계'})
)

In [177]:
jeonnam_6th.head(5)

,시도,구시군,선거인수,투표수,득표수_1_새누리당_이중효,득표수_2_새정치민주연합_이낙연,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수
0,전라남도,합계,1549440,1015688,92549,755233,120868,968650,47038,533752
1,전라남도,목포시,185570,103181,8017,80860,10840,99717,3464,82389
74,전라남도,여수시,232635,135390,11394,104927,15608,131929,3461,97245
159,전라남도,순천시,214889,126320,10695,95005,16777,122477,3843,88569
235,전라남도,나주시,76122,51515,4157,38556,6067,48780,2735,24607


In [178]:
jeonnam_6th.iloc[1:].select_dtypes(include='number').sum().to_frame().T

,선거인수,투표수,득표수_1_새누리당_이중효,득표수_2_새정치민주연합_이낙연,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수
0,1549440,1015688,92549,755233,120868,968650,47038,533752


In [179]:
jeonnam_6th_with_total = jeonnam_6th

In [180]:
jeonnam_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_이중효,득표수_2_새정치민주연합_이낙연,득표수_3_통합진보당_이성수,득표수_계,무효투표수,기권수
0,전라남도,합계,1549440,1015688,92549,755233,120868,968650,47038,533752
1,전라남도,목포시,185570,103181,8017,80860,10840,99717,3464,82389
74,전라남도,여수시,232635,135390,11394,104927,15608,131929,3461,97245
159,전라남도,순천시,214889,126320,10695,95005,16777,122477,3843,88569
235,전라남도,나주시,76122,51515,4157,38556,6067,48780,2735,24607
299,전라남도,광양시,115763,73537,8669,52699,9775,71143,2394,42226
339,전라남도,담양군,41046,29086,2627,22183,3004,27814,1272,11960
379,전라남도,장성군,39166,28305,3097,20673,3135,26905,1400,10861
416,전라남도,곡성군,26872,20627,1976,14497,2835,19308,1319,6245
453,전라남도,구례군,23363,19041,2351,12759,2796,17906,1135,4322


In [181]:
jeonnam_6th_with_total.to_csv("temp1_governor_jeonnam_6.csv", index=False, encoding="utf-8-sig")


## Gyeongbuk (2)


In [182]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_gyeongbuk = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EA%B2%BD%EC%83%81%EB%B6%81%EB%8F%84.xlsx'

gyeongbuk_6th = process_6th_governor_election2(
    file_path_or_url = blob_url6_gyeongbuk,
    header_rows = (4, 6),
    filter_column = '읍면동명',
    filter_value = [np.nan, '합계']
)

헤더 행: 4행 ~ 6행
생성된 컬럼 수: 13
데이터 행 수: 1093

생성된 컬럼명 (처음 10개):
0: 구시군별
1: 읍면동명
2: 구분
3: 선거인수
(가+나+다)
4: 투표수
(가+나)
5: 유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당
김관용
6: 새정치민주연합
오중기
7: 통합진보당
윤병태
8: 정의당
박창호
9: 계

'읍면동명' 컬럼 발견: 읍면동명
필터링: [nan, '합계'] 값들 중 하나인 행 추출
필터링 전: 1093행 → 필터링 후: 25행


In [183]:
gyeongbuk_6th

,구시군별,읍면동명,구분,선거인수\n(가+나+다),투표수\n(가+나),유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당\n김관용,새정치민주연합\n오중기,통합진보당\n윤병태,정의당\n박창호,계,무 효\n투표수\n(나),기권수\n(다),비고
0,경상북도,NaN,NaN,2211734,1314925,986989,189603,33458,59609,1269659,45266,896809,NaN
1,포항시북구,합계,NaN,216200,112449,82655,20064,2017,5017,109753,2696,103751,NaN
50,포항시남구,합계,NaN,201120,103003,76273,18395,2038,3840,100546,2457,98117,NaN
96,울릉군,합계,NaN,9344,7490,5925,847,158,242,7172,318,1854,NaN
109,경주시,합계,NaN,216922,126290,94824,16705,4915,5391,121835,4455,90632,NaN
182,김천시,합계,NaN,111552,70244,53436,9389,1401,3230,67456,2788,41308,NaN
252,안동시,합계,NaN,137715,90090,70575,11133,1884,3481,87073,3017,47625,NaN
328,구미시,합계,NaN,321096,160254,112367,32606,4259,6961,156193,4061,160842,NaN
413,영주시,합계,NaN,92496,62153,46595,8960,1215,3241,60011,2142,30343,NaN
474,영천시,합계,NaN,85998,54651,40283,7044,1243,3545,52115,2536,31347,NaN


In [184]:
gyeongbuk_6th.columns.tolist()

['구시군별',
 '읍면동명',
 '구분',
 '선거인수\n(가+나+다)',
 '투표수\n(가+나)',
 '유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당\n김관용',
 '새정치민주연합\n오중기',
 '통합진보당\n윤병태',
 '정의당\n박창호',
 '계',
 '무   효\n투표수\n(나)',
 '기권수\n(다)',
 '비고']

In [185]:
rename_gyeongbuk = {
    '구시군별': '구시군',
    '선거인수\n(가+나+다)': '선거인수',
    '투표수\n(가+나)': '투표수',
    '유 효 투 표 수 (가)_후 보 자 별 득 표 수_새누리당\n김관용': '득표수_1_새누리당_김관용',
    '새정치민주연합\n오중기': '득표수_2_새정치민주연합_오중기',
    '통합진보당\n윤병태': '득표수_3_통합진보당_윤병태',
    '정의당\n박창호': '득표수_4_정의당_박창호',
    '계': '득표수_계',
    '무   효\n투표수\n(나)': '무효투표수',
    '기권수\n(다)': '기권수'
    }

In [186]:
gyeongbuk_6th = gyeongbuk_6th.rename(columns=rename_gyeongbuk).drop(columns=['읍면동명', '구분', '비고'])
gyeongbuk_6th

,구시군,선거인수,투표수,득표수_1_새누리당_김관용,득표수_2_새정치민주연합_오중기,득표수_3_통합진보당_윤병태,득표수_4_정의당_박창호,득표수_계,무효투표수,기권수
0,경상북도,2211734,1314925,986989,189603,33458,59609,1269659,45266,896809
1,포항시북구,216200,112449,82655,20064,2017,5017,109753,2696,103751
50,포항시남구,201120,103003,76273,18395,2038,3840,100546,2457,98117
96,울릉군,9344,7490,5925,847,158,242,7172,318,1854
109,경주시,216922,126290,94824,16705,4915,5391,121835,4455,90632
182,김천시,111552,70244,53436,9389,1401,3230,67456,2788,41308
252,안동시,137715,90090,70575,11133,1884,3481,87073,3017,47625
328,구미시,321096,160254,112367,32606,4259,6961,156193,4061,160842
413,영주시,92496,62153,46595,8960,1215,3241,60011,2142,30343
474,영천시,85998,54651,40283,7044,1243,3545,52115,2536,31347


In [187]:
gyeongbuk_6th = gyeongbuk_6th.assign(
    시도='경상북도'
)[['시도'] + gyeongbuk_6th.columns.tolist()]

In [188]:
gyeongbuk_6th.loc[0, '구시군'] = '합계'

In [189]:
gyeongbuk_6th

,시도,구시군,선거인수,투표수,득표수_1_새누리당_김관용,득표수_2_새정치민주연합_오중기,득표수_3_통합진보당_윤병태,득표수_4_정의당_박창호,득표수_계,무효투표수,기권수
0,경상북도,합계,2211734,1314925,986989,189603,33458,59609,1269659,45266,896809
1,경상북도,포항시북구,216200,112449,82655,20064,2017,5017,109753,2696,103751
50,경상북도,포항시남구,201120,103003,76273,18395,2038,3840,100546,2457,98117
96,경상북도,울릉군,9344,7490,5925,847,158,242,7172,318,1854
109,경상북도,경주시,216922,126290,94824,16705,4915,5391,121835,4455,90632
182,경상북도,김천시,111552,70244,53436,9389,1401,3230,67456,2788,41308
252,경상북도,안동시,137715,90090,70575,11133,1884,3481,87073,3017,47625
328,경상북도,구미시,321096,160254,112367,32606,4259,6961,156193,4061,160842
413,경상북도,영주시,92496,62153,46595,8960,1215,3241,60011,2142,30343
474,경상북도,영천시,85998,54651,40283,7044,1243,3545,52115,2536,31347


In [190]:
gyeongbuk_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 0 to 1059
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 25 non-null     object
 1   구시군                25 non-null     object
 2   선거인수               25 non-null     object
 3   투표수                25 non-null     object
 4   득표수_1_새누리당_김관용     25 non-null     object
 5   득표수_2_새정치민주연합_오중기  25 non-null     object
 6   득표수_3_통합진보당_윤병태    25 non-null     object
 7   득표수_4_정의당_박창호      25 non-null     object
 8   득표수_계              25 non-null     object
 9   무효투표수              25 non-null     object
 10  기권수                25 non-null     object
dtypes: object(11)
memory usage: 2.9+ KB


In [191]:
gyeongbuk_6th = gyeongbuk_6th.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
)

In [192]:
gyeongbuk_6th_with_total = gyeongbuk_6th

In [193]:
gyeongbuk_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_김관용,득표수_2_새정치민주연합_오중기,득표수_3_통합진보당_윤병태,득표수_4_정의당_박창호,득표수_계,무효투표수,기권수
0,경상북도,합계,2211734,1314925,986989,189603,33458,59609,1269659,45266,896809
1,경상북도,포항시북구,216200,112449,82655,20064,2017,5017,109753,2696,103751
50,경상북도,포항시남구,201120,103003,76273,18395,2038,3840,100546,2457,98117
96,경상북도,울릉군,9344,7490,5925,847,158,242,7172,318,1854
109,경상북도,경주시,216922,126290,94824,16705,4915,5391,121835,4455,90632
182,경상북도,김천시,111552,70244,53436,9389,1401,3230,67456,2788,41308
252,경상북도,안동시,137715,90090,70575,11133,1884,3481,87073,3017,47625
328,경상북도,구미시,321096,160254,112367,32606,4259,6961,156193,4061,160842
413,경상북도,영주시,92496,62153,46595,8960,1215,3241,60011,2142,30343
474,경상북도,영천시,85998,54651,40283,7044,1243,3545,52115,2536,31347


In [194]:
gyeongbuk_6th_with_total.to_csv("temp1_governor_gyeongbuk_6.csv", index=False, encoding="utf-8-sig")


## Gyeongnam (1)


In [195]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_gyeongnam = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EA%B2%BD%EC%83%81%EB%82%A8%EB%8F%84.xls'

gyeongnam_6th = process_6th_governor_election1(
    file_path_or_url = blob_url6_gyeongnam,
    header_rows = (2, 3),
    filter_column = '읍면동명',
    filter_value = '합계'
)

헤더 행: 2행 ~ 3행
생성된 컬럼 수: 11
데이터 행 수: 1034

생성된 컬럼명 (처음 10개):
0: 구시군
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
홍준표
6: 새정치민주연합
김경수
7: 통합진보당
강병기
8: 계
9: 무효
투표수

'읍면동명' 컬럼 발견: 읍면동명
필터링 전: 1034행 → 필터링 후: 23행
'읍면동명' == '합계'인 행만 추출


In [196]:
gyeongnam_6th

,구시군,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n홍준표,새정치민주연합\n김경수,통합진보당\n강병기,계,무효\n투표수,기권수
0,경상남도,합계,NaN,2658347,1589673,913162,559367,79015,1551544,38129,1068674
1,창원시의창구,합계,NaN,204523,114829,59480,47216,6020,112716,2113,89694
29,창원시성산구,합계,NaN,185723,111830,50723,52430,7033,110186,1644,73893
54,창원시마산합포구,합계,NaN,151800,87780,58522,24933,2756,86211,1569,64020
115,창원시마산회원구,합계,NaN,175918,102205,63561,33392,3673,100626,1579,73713
158,창원시진해구,합계,NaN,139286,77939,45520,27864,3079,76463,1476,61347
207,진주시,합계,NaN,268027,165372,99871,50316,11916,162103,3269,102655
307,통영시,합계,NaN,111561,67447,43751,18819,3146,65716,1731,44114
356,고성군,합계,NaN,47887,32026,19165,10011,1705,30881,1145,15861
402,사천시,합계,NaN,94250,63316,40192,17624,3469,61285,2031,30934


In [197]:
gyeongnam_6th.columns.tolist()

['구시군',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n홍준표',
 '새정치민주연합\n김경수',
 '통합진보당\n강병기',
 '계',
 '무효\n투표수',
 '기권수']

In [198]:
rename_gyeongnam = {
    '후보자별 득표수_새누리당\n홍준표': '득표수_1_새누리당_홍준표',
    '새정치민주연합\n김경수': '득표수_2_새정치민주연합_김경수',
    '통합진보당\n강병기': '득표수_3_통합진보당_강병기',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [199]:
gyeongnam_6th = gyeongnam_6th.rename(columns=rename_gyeongnam).drop(columns=['읍면동명', '구분'])
gyeongnam_6th

,구시군,선거인수,투표수,득표수_1_새누리당_홍준표,득표수_2_새정치민주연합_김경수,득표수_3_통합진보당_강병기,득표수_계,무효투표수,기권수
0,경상남도,2658347,1589673,913162,559367,79015,1551544,38129,1068674
1,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694
29,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893
54,창원시마산합포구,151800,87780,58522,24933,2756,86211,1569,64020
115,창원시마산회원구,175918,102205,63561,33392,3673,100626,1579,73713
158,창원시진해구,139286,77939,45520,27864,3079,76463,1476,61347
207,진주시,268027,165372,99871,50316,11916,162103,3269,102655
307,통영시,111561,67447,43751,18819,3146,65716,1731,44114
356,고성군,47887,32026,19165,10011,1705,30881,1145,15861
402,사천시,94250,63316,40192,17624,3469,61285,2031,30934


In [200]:
gyeongnam_6th = gyeongnam_6th.assign(
    시도='경상남도'
)[['시도'] + gyeongnam_6th.columns.tolist()]

In [201]:
gyeongnam_6th.head()

,시도,구시군,선거인수,투표수,득표수_1_새누리당_홍준표,득표수_2_새정치민주연합_김경수,득표수_3_통합진보당_강병기,득표수_계,무효투표수,기권수
0,경상남도,경상남도,2658347,1589673,913162,559367,79015,1551544,38129,1068674
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694
29,경상남도,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893
54,경상남도,창원시마산합포구,151800,87780,58522,24933,2756,86211,1569,64020
115,경상남도,창원시마산회원구,175918,102205,63561,33392,3673,100626,1579,73713


In [202]:
gyeongnam_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 0 to 979
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 23 non-null     object
 1   구시군                23 non-null     object
 2   선거인수               23 non-null     object
 3   투표수                23 non-null     object
 4   득표수_1_새누리당_홍준표     23 non-null     object
 5   득표수_2_새정치민주연합_김경수  23 non-null     object
 6   득표수_3_통합진보당_강병기    23 non-null     object
 7   득표수_계              23 non-null     object
 8   무효투표수              23 non-null     object
 9   기권수                23 non-null     object
dtypes: object(10)
memory usage: 2.5+ KB


In [203]:
gyeongnam_6th = gyeongnam_6th.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
).assign(
    구시군=lambda df: df['구시군']
        .replace({'경상남도': '합계'})
)

In [204]:
gyeongnam_6th.head(5)

,시도,구시군,선거인수,투표수,득표수_1_새누리당_홍준표,득표수_2_새정치민주연합_김경수,득표수_3_통합진보당_강병기,득표수_계,무효투표수,기권수
0,경상남도,합계,2658347,1589673,913162,559367,79015,1551544,38129,1068674
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694
29,경상남도,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893
54,경상남도,창원시마산합포구,151800,87780,58522,24933,2756,86211,1569,64020
115,경상남도,창원시마산회원구,175918,102205,63561,33392,3673,100626,1579,73713


In [205]:
gyeongnam_6th_with_total = gyeongnam_6th

In [206]:
gyeongnam_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_홍준표,득표수_2_새정치민주연합_김경수,득표수_3_통합진보당_강병기,득표수_계,무효투표수,기권수
0,경상남도,합계,2658347,1589673,913162,559367,79015,1551544,38129,1068674
1,경상남도,창원시의창구,204523,114829,59480,47216,6020,112716,2113,89694
29,경상남도,창원시성산구,185723,111830,50723,52430,7033,110186,1644,73893
54,경상남도,창원시마산합포구,151800,87780,58522,24933,2756,86211,1569,64020
115,경상남도,창원시마산회원구,175918,102205,63561,33392,3673,100626,1579,73713
158,경상남도,창원시진해구,139286,77939,45520,27864,3079,76463,1476,61347
207,경상남도,진주시,268027,165372,99871,50316,11916,162103,3269,102655
307,경상남도,통영시,111561,67447,43751,18819,3146,65716,1731,44114
356,경상남도,고성군,47887,32026,19165,10011,1705,30881,1145,15861
402,경상남도,사천시,94250,63316,40192,17624,3469,61285,2031,30934


In [207]:
gyeongnam_6th_with_total.to_csv("temp1_governor_gyeongnam_6.csv", index=False, encoding="utf-8-sig")


## Jeju (2)

In [208]:
# 깃허브 blob URL로 불러오는 경우 (자동으로 raw URL로 변환됨)
blob_url6_jeju = 'https://github.com/sw1kwon/korean-elections/blob/main/original/Local_Elections_Governor/6th_2014/%EC%A0%9C%EC%A3%BC%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84_%EB%8F%84%EC%A7%80%EC%82%AC.xls'

jeju_6th = process_6th_governor_election2(
    file_path_or_url = blob_url6_jeju,
    header_rows = (3, 4),
    filter_column = '읍면동명',
    filter_value = [np.nan, '소계']
)

헤더 행: 3행 ~ 4행
생성된 컬럼 수: 12
데이터 행 수: 138

생성된 컬럼명 (처음 10개):
0: 구시군명
1: 읍면동명
2: 구분
3: 선거인수
4: 투표수
5: 후보자별 득표수_새누리당
원희룡
6: 새정치민주연합
신구범
7: 통합진보당
고승완
8: 새정치당
주종근
9: 계

'읍면동명' 컬럼 발견: 읍면동명
필터링: [nan, '소계'] 값들 중 하나인 행 추출
필터링 전: 138행 → 필터링 후: 3행


In [209]:
jeju_6th

,구시군명,읍면동명,구분,선거인수,투표수,후보자별 득표수_새누리당\n원희룡,새정치민주연합\n신구범,통합진보당\n고승완,새정치당\n주종근,계,무효\n투표수,기권수
0,제주특별자치도,NaN,NaN,467182,293323,172793,99493,12209,3637,288132,5191,173859
1,제주시,소계,NaN,340604,208224,119758,73632,9101,2213,204704,3520,132380
83,서귀포시,소계,NaN,126578,85099,53035,25861,3108,1424,83428,1671,41479


In [210]:
jeju_6th.columns.tolist()

['구시군명',
 '읍면동명',
 '구분',
 '선거인수',
 '투표수',
 '후보자별 득표수_새누리당\n원희룡',
 '새정치민주연합\n신구범',
 '통합진보당\n고승완',
 '새정치당\n주종근',
 '계',
 '무효\n투표수',
 '기권수']

In [211]:
rename_jeju = {
    '구시군명': '구시군',
    '후보자별 득표수_새누리당\n원희룡': '득표수_1_새누리당_원희룡',
    '새정치민주연합\n신구범': '득표수_2_새정치민주연합_신구범',
    '통합진보당\n고승완': '득표수_3_통합진보당_고승완',
    '새정치당\n주종근': '득표수_4_새정치국민의당_주종근',
    '계': '득표수_계',
    '무효\n투표수': '무효투표수'
    }

In [212]:
jeju_6th = jeju_6th.rename(columns=rename_jeju).drop(columns=['읍면동명', '구분'])
jeju_6th

,구시군,선거인수,투표수,득표수_1_새누리당_원희룡,득표수_2_새정치민주연합_신구범,득표수_3_통합진보당_고승완,득표수_4_새정치국민의당_주종근,득표수_계,무효투표수,기권수
0,제주특별자치도,467182,293323,172793,99493,12209,3637,288132,5191,173859
1,제주시,340604,208224,119758,73632,9101,2213,204704,3520,132380
83,서귀포시,126578,85099,53035,25861,3108,1424,83428,1671,41479


In [213]:
jeju_6th = jeju_6th.assign(
    시도='제주특별자치도'
)[['시도'] + jeju_6th.columns.tolist()]

In [214]:
jeju_6th

,시도,구시군,선거인수,투표수,득표수_1_새누리당_원희룡,득표수_2_새정치민주연합_신구범,득표수_3_통합진보당_고승완,득표수_4_새정치국민의당_주종근,득표수_계,무효투표수,기권수
0,제주특별자치도,제주특별자치도,467182,293323,172793,99493,12209,3637,288132,5191,173859
1,제주특별자치도,제주시,340604,208224,119758,73632,9101,2213,204704,3520,132380
83,제주특별자치도,서귀포시,126578,85099,53035,25861,3108,1424,83428,1671,41479


In [215]:
jeju_6th.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 0 to 83
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   시도                 3 non-null      object
 1   구시군                3 non-null      object
 2   선거인수               3 non-null      object
 3   투표수                3 non-null      object
 4   득표수_1_새누리당_원희룡     3 non-null      object
 5   득표수_2_새정치민주연합_신구범  3 non-null      object
 6   득표수_3_통합진보당_고승완    3 non-null      object
 7   득표수_4_새정치국민의당_주종근  3 non-null      object
 8   득표수_계              3 non-null      object
 9   무효투표수              3 non-null      object
 10  기권수                3 non-null      object
dtypes: object(11)
memory usage: 396.0+ bytes


In [216]:
jeju_6th = jeju_6th.apply(
    lambda col: col.astype(int)
    if col.dtype == 'object' and col.astype(str).str.fullmatch(r'\d+').all()
    else col
).assign(
    구시군=lambda df: df['구시군']
        .replace({'제주특별자치도': '합계'})
)

In [217]:
jeju_6th

,시도,구시군,선거인수,투표수,득표수_1_새누리당_원희룡,득표수_2_새정치민주연합_신구범,득표수_3_통합진보당_고승완,득표수_4_새정치국민의당_주종근,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,467182,293323,172793,99493,12209,3637,288132,5191,173859
1,제주특별자치도,제주시,340604,208224,119758,73632,9101,2213,204704,3520,132380
83,제주특별자치도,서귀포시,126578,85099,53035,25861,3108,1424,83428,1671,41479


In [218]:
jeju_6th_with_total = jeju_6th

In [219]:
jeju_6th_with_total

,시도,구시군,선거인수,투표수,득표수_1_새누리당_원희룡,득표수_2_새정치민주연합_신구범,득표수_3_통합진보당_고승완,득표수_4_새정치국민의당_주종근,득표수_계,무효투표수,기권수
0,제주특별자치도,합계,467182,293323,172793,99493,12209,3637,288132,5191,173859
1,제주특별자치도,제주시,340604,208224,119758,73632,9101,2213,204704,3520,132380
83,제주특별자치도,서귀포시,126578,85099,53035,25861,3108,1424,83428,1671,41479


In [220]:
jeju_6th_with_total.to_csv("temp1_governor_jeju_6.csv", index=False, encoding="utf-8-sig")

# Batch CSV Files to ZIP

In [221]:
import zipfile
import glob

# Find all CSV files in current directory
csv_files = glob.glob('*.csv')

# Create ZIP file
with zipfile.ZipFile('all_csv_files.zip', 'w') as zipf:
   for file in csv_files:
       zipf.write(file)
       print(f"Added: {file}")  # Show progress

print(f"Total {len(csv_files)} files compressed.")

Added: temp1_governor_incheon_6.csv
Added: temp1_governor_gangwon_6.csv
Added: temp1_governor_gwangju_6.csv
Added: temp1_governor_jeonbuk_6.csv
Added: temp1_governor_gyeongbuk_6.csv
Added: temp1_governor_chungnam_6.csv
Added: temp1_governor_seoul_6.csv
Added: temp1_governor_gyeonggi_6.csv
Added: temp1_governor_busan_6.csv
Added: temp1_governor_daegu_6.csv
Added: temp1_governor_jeju_6.csv
Added: temp1_governor_jeonnam_6.csv
Added: temp1_governor_daejeon_6.csv
Added: temp1_governor_chungbuk_6.csv
Added: temp1_governor_gyeongnam_6.csv
Added: temp1_governor_sejong_6.csv
Added: temp1_governor_ulsan_6.csv
Total 17 files compressed.
